In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ============================================================
# Setup & Environment
# ============================================================
import os
workspace_path = "/content/drive/MyDrive/Efficient-Financial-NLP-Fine-Tuning-with-QLoRA"
os.makedirs(workspace_path, exist_ok=True)
os.chdir(workspace_path)
!pwd

/content/drive/MyDrive/Efficient-Financial-NLP-Fine-Tuning-with-QLoRA


In [ ]:
from huggingface_hub import login
login(token="secret token here")

# ***CELL 1: GLOBAL CONFIGURATION & PROJECT SETUP***



In [ ]:
# ***CELL 1: GLOBAL CONFIGURATION & PROJECT SETUP***

"""
Defines project structure and global configurations
"""

print("="*80)
print("STEP 1: PROJECT SETUP")
print("="*80 + "\n")

import os
from pathlib import Path

# ============================================================================
# 1.1: CREATE PROJECT STRUCTURE
# ============================================================================

print("Creating directory structure...")

directories = [
    "configs/tasks",
    "src/data",
    "src/models",
    "src/training",
    "src/evaluation",
    "src/utils",
    "scripts",
    "data/raw",
    "data/formatted",
    "outputs/adapters",
    "outputs/evaluations",
    "outputs/logs",
    "logs"
]

for directory in directories:
    Path(directory).mkdir(parents=True, exist_ok=True)
    print(f"  [+] {directory}")

# Create __init__.py files
for root in ["src", "src/data", "src/models", "src/training", "src/evaluation", "src/utils"]:
    init_file = Path(root) / "__init__.py"
    init_file.touch()

print("\n[OK] Directory structure created\n")


# ============================================================================
# 1.2: DATASET MAPPING (SOURCE → TASK)
# ============================================================================

DATASET_CONFIG = {
    # Task: Sentiment Analysis
    "sa": {
        "task_name": "Sentiment Analysis",
        "datasets": {
            "fpb": {
                "hf_path": "ChanceFocus/en-fpb",
                "description": "Financial Phrase Bank"
            },
            "fiqasa": {
                "hf_path": "ChanceFocus/flare-fiqasa",
                "description": "FiQA Sentiment Analysis"
            }
        },
        "system_prompt": None,  # Extracted from query
        "labels": ["negative", "neutral", "positive"],
        "max_length": 512,
        "use_dynamic_system_prompt": True,
        "system_prompt_delimiter": "Text:"
    },

    # Task: Headline Classification
    "hc": {
        "task_name": "Headline Classification",
        "datasets": {
            "headlines": {
                "hf_path": "ChanceFocus/flare-headlines",
                "description": "Gold Headlines Classification"
            }
        },
        "system_prompt": None,  # Will be extracted from query
        "labels": ["no", "yes"],
        "max_length": 128,
        "use_dynamic_system_prompt": True,
        "system_prompt_delimiter": "Text:"
    },

    # Task: Named Entity Recognition
    "ner": {
        "task_name": "Named Entity Recognition",
        "datasets": {
            "flare_ner": {
                "hf_path": "TheFinAI/flare-ner",
                "description": "FLARE NER Dataset"
            }
        },
        "system_prompt": None,  # Will be extracted from query
        "labels": ["PER", "ORG", "LOC"],
        "max_length": 1024,
        "use_dynamic_system_prompt": True,
        "system_prompt_delimiter": "Text:"
    },

    # Task: Question Answering
    "qa": {
        "task_name": "Question Answering",
        "datasets": {
            "finqa": {
                "hf_path": "ChanceFocus/flare-finqa",
                "description": "Financial Question Answering",
                "is_conversational": False
            },
            "convfinqa": {
                "hf_path": "ChanceFocus/flare-convfinqa",
                "description": "Conversational FinQA",
                "is_conversational": True
            }
        },
        "system_prompt": None,  # Extracted from query
        "labels": None,
        "max_length": 2048,
        "use_dynamic_system_prompt": True,
        "conversational_config": {
            "extract_system_from_query": True,
            "system_delimiter": "Context:",
            "conversation_marker": "Conversations:",
            "question_marker": "Question:",
            "parse_conversation_from_query": True
        }
    },

    # Task: Stock Movement Prediction
    "smp": {
        "task_name": "Stock Movement Prediction",
        "datasets": {
            "stock_cikm": {
                "hf_path": "ChanceFocus/flare-sm-cikm",
                "description": "Stock Movement (CIKM)"
            },
            "stock_bigdata": {
                "hf_path": "TheFinAI/flare-sm-bigdata",
                "description": "Stock Movement (BigData)"
            }
        },
        "system_prompt": None,  # Will be extracted from query
        "labels": ["fall", "rise"],
        "max_length": 2048,
        "max_samples_per_source": 10000,
        "use_dynamic_system_prompt": True,
        "system_prompt_delimiter": "Context:"
    }
}


# Save config
import json
with open("data/dataset_config.json", "w") as f:
    json.dump(DATASET_CONFIG, f, indent=2)

print("Dataset configuration:")
for task, config in DATASET_CONFIG.items():
    print(f"\n  {task.upper()}: {config['task_name']}")
    print(f"    Datasets: {len(config['datasets'])}")
    print(f"    Dynamic System Prompt: {config.get('use_dynamic_system_prompt', False)}")
    for ds_name, ds_info in config['datasets'].items():
        print(f"      - {ds_name}: {ds_info['description']}")

print("\n[OK] Configuration saved to data/dataset_config.json\n")


# ============================================================================
# 1.3: LLAMA TEMPLATE
# ============================================================================

LLAMA_TEMPLATE = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{assistant}<|eot_id|>"""

with open("data/llama_template.txt", "w") as f:
    f.write(LLAMA_TEMPLATE)

print("[OK] Llama template saved\n")

print("="*80)
print("[OK] PROJECT SETUP COMPLETE")
print("="*80 + "\n")

STEP 1: PROJECT SETUP

Creating directory structure...
  [+] configs/tasks
  [+] src/data
  [+] src/models
  [+] src/training
  [+] src/evaluation
  [+] src/utils
  [+] scripts
  [+] data/raw
  [+] data/formatted
  [+] outputs/adapters
  [+] outputs/evaluations
  [+] outputs/logs
  [+] logs

[OK] Directory structure created

Dataset configuration:

  SA: Sentiment Analysis
    Datasets: 2
    Dynamic System Prompt: True
      - fpb: Financial Phrase Bank
      - fiqasa: FiQA Sentiment Analysis

  HC: Headline Classification
    Datasets: 1
    Dynamic System Prompt: True
      - headlines: Gold Headlines Classification

  NER: Named Entity Recognition
    Datasets: 1
    Dynamic System Prompt: True
      - flare_ner: FLARE NER Dataset

  QA: Question Answering
    Datasets: 2
    Dynamic System Prompt: True
      - finqa: Financial Question Answering
      - convfinqa: Conversational FinQA

  SMP: Stock Movement Prediction
    Datasets: 2
    Dynamic System Prompt: True
      - stock_c

# ***CELL 2: DATASET FORMATTER***

In [ ]:
# ***CELL 2: DATASET FORMATTER***

"""
Formats datasets into Llama 3.1 format
"""

from datasets import load_dataset, concatenate_datasets, DatasetDict
import json
import re
from pathlib import Path
from tqdm import tqdm
import random
from collections import OrderedDict

print("="*80)
print("STEP 2: DATASET FORMATTING")
print("="*80 + "\n")

with open("data/dataset_config.json", "r") as f:
    DATASET_CONFIG = json.load(f)

with open("data/llama_template.txt", "r") as f:
    LLAMA_TEMPLATE = f.read()

print("[OK] Configuration loaded\n")


# ============================================================================
# 2.1: TEXT NORMALIZATION
# ============================================================================

def clean_text(text):
    """Clean and normalize text with UTF-8 fix"""
    if not text:
        return ""

    if isinstance(text, str):
        try:
            text = text.replace('�', '£')
            text = text.replace('\ufffd', '')
            text = text.encode('utf-8', errors='ignore').decode('utf-8', errors='ignore')
        except:
            text = str(text)

    text = re.sub(r'\s+', ' ', str(text)).strip()
    return text


def normalize_label(text, labels):
    """
    Normalize labels to lowercase for consistency
    All labels are now lowercase
    """
    if not text:
        return None

    text_clean = str(text).strip().lower()

    if text_clean in labels:
        return text_clean

    if "positive" in labels and "negative" in labels and "neutral" in labels:
        if text_clean in ['pos', 'bullish']:
            return "positive"
        elif text_clean in ['neg', 'bearish']:
            return "negative"
        elif text_clean in ['neut']:
            return "neutral"

    elif "yes" in labels and "no" in labels:
        if text_clean in ['y', 'true', '1']:
            return "yes"
        elif text_clean in ['n', 'false', '0']:
            return "no"

    elif "rise" in labels and "fall" in labels:
        if text_clean in ['up', 'increase', 'gain']:
            return "rise"
        elif text_clean in ['down', 'decrease', 'loss']:
            return "fall"

    return None


def extract_system_and_text(query_text, marker="Text:"):
    """
    Extract system prompt and text from query
    Format: [SYSTEM PROMPT] Text: [actual text]
    Returns: (system_prompt, text)
    """
    if not query_text or marker not in query_text:
        return None, None

    parts = query_text.split(marker, 1)
    system_prompt = parts[0].strip()
    text = parts[1].strip() if len(parts) > 1 else None

    if text:
        text = text.split('Answer:')[0].strip()

    return system_prompt, text


def clean_ner_answer(answer_text):
    """
    Clean NER answer: remove duplicates and normalize case
    """
    if not answer_text:
        return None

    lines = answer_text.strip().split('\n')
    entities_dict = OrderedDict()

    for line in lines:
        line = line.strip()

        if not line or ',' not in line:
            continue

        parts = line.rsplit(',', 1)

        if len(parts) != 2:
            continue

        entity_name = parts[0].strip()
        entity_type = parts[1].strip().upper()

        if entity_type not in ['PER', 'ORG', 'LOC']:
            continue

        if not entity_name:
            continue

        key = (entity_name.lower(), entity_type)

        if key not in entities_dict:
            entities_dict[key] = entity_name

    if not entities_dict:
        return None

    cleaned_lines = [
        f"{entity_name}, {entity_type}"
        for (entity_name_lower, entity_type), entity_name in entities_dict.items()
    ]

    return '\n'.join(cleaned_lines)


# ============================================================================
# 2.2: TASK-SPECIFIC FORMATTERS
# ============================================================================

def format_sentiment_analysis(example, labels):
    """
    Format SA task - Extract system prompt from query, text from text column
    """
    if 'query' not in example or not example['query']:
        return None

    system_prompt, _ = extract_system_and_text(example['query'], marker="Text:")

    if not system_prompt:
        return None

    text = clean_text(example.get('text', ''))

    if not text and 'query' in example:
        _, text_from_query = extract_system_and_text(example['query'], marker="Text:")
        text = clean_text(text_from_query) if text_from_query else None

    if not text or len(text) < 10:
        return None

    label_raw = None
    for field in ['answer', 'label', 'sentiment', 'target']:
        if field in example and example[field] is not None:
            label_raw = example[field]
            break

    if label_raw is None:
        return None

    label = normalize_label(label_raw, labels)
    if not label:
        return None

    return {
        "text": LLAMA_TEMPLATE.format(
            system=system_prompt,
            user=text,
            assistant=label
        )
    }


def format_headline_classification(example, labels):
    """
    Format HC task - Extract system prompt from query
    """
    if 'query' not in example or not example['query']:
        return None

    system_prompt, text = extract_system_and_text(example['query'], marker="Text:")

    if not system_prompt or not text:
        return None

    text = clean_text(text)

    if not text or len(text) < 10:
        return None

    label_raw = None
    for field in ['answer', 'label']:
        if field in example and example[field] is not None:
            label_raw = example[field]
            break

    if label_raw is None:
        return None

    label = normalize_label(label_raw, labels)
    if not label:
        return None

    return {
        "text": LLAMA_TEMPLATE.format(
            system=system_prompt,
            user=text,
            assistant=label
        )
    }


def format_ner(example):
    """
    Format NER task - Extract system prompt from query, text from text column
    """
    if 'query' not in example or not example['query']:
        return None

    if 'text' not in example or not example['text']:
        return None

    system_prompt, _ = extract_system_and_text(example['query'], marker="Text:")

    if not system_prompt:
        return None

    text = clean_text(example['text'])

    if not text or len(text) < 20:
        return None

    entities = example.get("answer", "")

    if isinstance(entities, list):
        formatted = []
        for entity in entities:
            if isinstance(entity, dict):
                name = entity.get("name", entity.get("text", ""))
                etype = entity.get("type", entity.get("label", ""))
                if name and etype:
                    formatted.append(f"{name}, {etype}")
        answer = "\n".join(formatted) if formatted else None
    else:
        answer = str(entities).strip() if entities else None

    if not answer:
        return None

    answer_clean = clean_ner_answer(answer)

    if not answer_clean:
        return None

    return {
        "text": LLAMA_TEMPLATE.format(
            system=system_prompt,
            user=text,
            assistant=answer_clean
        )
    }


# ============================================================================
# 2.3: QA FORMATTERS
# ============================================================================

def parse_convfinqa_conversations(query_text):
    """
    Parse ConvFinQA conversations from query
    Format: "Conversations: q0: question answer q1: question answer"
    Returns: formatted conversation history string
    """
    if not query_text or "Conversations:" not in query_text:
        return ""

    conv_match = re.search(r'Conversations:(.*?)Question:', query_text, re.DOTALL)
    if not conv_match:
        return ""

    conv_text = conv_match.group(1).strip()

    pattern = r'q(\d+):\s*(.+?)(?=q\d+:|$)'
    matches = re.findall(pattern, conv_text, re.DOTALL)

    if not matches:
        return ""

    history_parts = []
    for turn_num, qa_content in matches:
        qa_content = qa_content.strip()

        parts = qa_content.rsplit(maxsplit=1)

        if len(parts) == 2:
            question = parts[0].strip()
            answer = parts[1].strip()
            history_parts.append(f"Q{turn_num}: {question}\nA{turn_num}: {answer}")
        else:
            history_parts.append(f"Q{turn_num}: {qa_content}")

    return "\n\n".join(history_parts) if history_parts else ""


def format_finqa(example):
    """
    Format FinQA - Context from query, question from text column
    """
    if 'query' not in example or not example['query']:
        return None

    query_text = example['query']

    system_prompt = None
    if "Context:" in query_text:
        system_prompt = query_text.split("Context:", 1)[0].strip()

    if not system_prompt:
        return None

    context = None
    if "Context:" in query_text:
        context_part = query_text.split("Context:", 1)[1]
        if "Question:" in context_part:
            context = context_part.split("Question:", 1)[0].strip()
        else:
            context = context_part.strip()

    context = clean_text(context) if context else None

    if not context:
        return None

    question = clean_text(example.get('text', ''))

    if not question:
        return None

    user_text = f"Context:\n{context}\n\nQuestion: {question}"

    answer = example.get("answer", "")

    if isinstance(answer, (int, float)):
        answer = str(answer)
    elif isinstance(answer, list):
        answer = ", ".join(str(x) for x in answer)
    else:
        answer = str(answer).strip()

    if not answer:
        return None

    return {
        "text": LLAMA_TEMPLATE.format(
            system=system_prompt,
            user=user_text,
            assistant=answer
        )
    }


def format_convfinqa(example):
    """
    Format ConvFinQA (conversational) - System prompt from query
    """
    if 'query' not in example or not example['query']:
        return None

    query_text = example['query']

    system_prompt = None
    if "Context:" in query_text:
        system_prompt = query_text.split("Context:", 1)[0].strip()

    if not system_prompt:
        return None

    question = None
    if "Question:" in query_text:
        question = query_text.split("Question:", 1)[1].strip()
        question = re.sub(r'\s*Answer:\s*$', '', question, flags=re.IGNORECASE).strip()

    if not question:
        return None

    context = None
    if "Context:" in query_text:
        context_part = query_text.split("Context:", 1)[1]
        if "Conversations:" in context_part:
            context = context_part.split("Conversations:", 1)[0].strip()
        elif "Question:" in context_part:
            context = context_part.split("Question:", 1)[0].strip()
        else:
            context = context_part.strip()

    context = clean_text(context) if context else None

    if not context:
        return None

    turn = example.get('turn', 0)
    conversation_history = ""

    if turn and int(turn) > 0:
        conversation_history = parse_convfinqa_conversations(query_text)

    user_parts = [f"Context:\n{context}"]

    if conversation_history:
        user_parts.append(f"\nPrevious conversation:\n{conversation_history}")

    user_parts.append(f"\nCurrent question: {question}")

    user_text = "\n".join(user_parts)

    answer = example.get("answer", "")

    if isinstance(answer, (int, float)):
        answer = str(answer)
    elif isinstance(answer, list):
        answer = ", ".join(str(x) for x in answer)
    else:
        answer = str(answer).strip()

    if not answer:
        return None

    return {
        "text": LLAMA_TEMPLATE.format(
            system=system_prompt,
            user=user_text,
            assistant=answer
        )
    }


def extract_target_info(query_text):
    """
    Extract target ticker and date from query
    Example: "predict closing price of $c at 2017-01-18"
    Returns: (ticker, date)
    """
    ticker = None
    date = None

    ticker_match = re.search(r'\$([a-z]+)\b', query_text, re.IGNORECASE)
    if ticker_match:
        ticker = ticker_match.group(1).upper()

    date_match = re.search(r'(\d{4}-\d{2}-\d{2})', query_text)
    if date_match:
        date = date_match.group(1)

    return ticker, date


def format_stock_movement(example, labels):
    """
    Format SMP task - Extract system prompt from query, context from text column
    """
    if 'query' not in example or not example['query']:
        return None

    if 'text' not in example or not example['text']:
        return None

    query_text = example['query']

    system_prompt = None
    if 'Context:' in query_text:
        system_prompt = query_text.split('Context:', 1)[0].strip()

    if not system_prompt:
        return None

    raw_data = clean_text(example['text'])

    if not raw_data:
        return None

    ticker, target_date = extract_target_info(query_text)

    if not ticker:
        ticker = example.get("tid", example.get("ticker", ""))
    if not target_date:
        target_date = example.get("point", example.get("date", ""))

    lines = [l.strip() for l in raw_data.split('\n') if l.strip()]

    csv_section = []
    tweets_section = []
    in_csv = False

    for line in lines:
        if 'date' in line.lower() and ('open' in line.lower() or 'close' in line.lower()):
            in_csv = True
            csv_section.append(line)
        elif in_csv and re.match(r'^\d{4}-\d{2}-\d{2}', line):
            csv_section.append(line)
        elif in_csv and not line.count(',') >= 5:
            in_csv = False
            if len(line) > 10:
                tweets_section.append(line)
        elif re.match(r'^\d{4}-\d{2}-\d{2}:', line):
            tweets_section.append(line)
        elif not in_csv and len(line) > 10:
            tweets_section.append(line)

    parts = []

    if ticker and target_date:
        parts.append(f"Predict price movement for ${ticker} on {target_date}")

    if csv_section:
        csv_text = '\n'.join(csv_section)
        parts.append(f"\nHistorical Price Data:\n{csv_text}")

    if tweets_section:
        tweets_text = '\n'.join(tweets_section)
        parts.append(f"\nSocial Media Sentiment:\n{tweets_text}")

    if len(parts) <= 1:
        return None

    user_text = "\n".join(parts)

    label = normalize_label(example.get("answer", ""), labels)

    if not label:
        return None

    return {
        "text": LLAMA_TEMPLATE.format(
            system=system_prompt,
            user=user_text,
            assistant=label
        )
    }


# ============================================================================
# 2.4: MAIN FORMATTING FUNCTION
# ============================================================================

def format_dataset(task_key, dataset_key, hf_path, config, dataset_info):
    """Format a single dataset"""
    print(f"\n  Loading {dataset_key} from {hf_path}...")

    try:
        dataset = load_dataset(hf_path)

        labels = config.get('labels')

        if task_key == "ner":
            formatter = lambda ex: format_ner(ex)
        elif task_key == "hc":
            formatter = lambda ex: format_headline_classification(ex, labels)
        elif task_key == "sa":
            formatter = lambda ex: format_sentiment_analysis(ex, labels)
        elif task_key == "smp":
            formatter = lambda ex: format_stock_movement(ex, labels)
        elif task_key == "qa":
            is_conversational = dataset_info.get('is_conversational', False)
            if is_conversational:
                formatter = lambda ex: format_convfinqa(ex)
                print(f"  Using CONVERSATIONAL formatter (ConvFinQA)")
            else:
                formatter = lambda ex: format_finqa(ex)
                print(f"  Using STANDARD formatter (FinQA)")

        def safe_format(example):
            try:
                result = formatter(example)
                return result if result else {"text": None}
            except Exception as e:
                return {"text": None}

        print(f"  Formatting...")
        formatted = dataset.map(
            safe_format,
            remove_columns=dataset['train'].column_names,
            desc=f"Formatting {dataset_key}"
        )

        for split in formatted.keys():
            before = len(formatted[split])
            formatted[split] = formatted[split].filter(lambda x: x['text'] is not None)
            after = len(formatted[split])
            if before > after:
                print(f"  [WARN] Filtered {before - after} invalid examples from {split}")

        max_samples = config.get('max_samples_per_source')
        if max_samples:
            for split in formatted.keys():
                if len(formatted[split]) > max_samples:
                    indices = random.sample(range(len(formatted[split])), max_samples)
                    formatted[split] = formatted[split].select(indices)
                    print(f"  Sampled {max_samples} from {split}")

        for split in formatted.keys():
            print(f"    {split}: {len(formatted[split]):,} samples")

        return DatasetDict(formatted)

    except Exception as e:
        print(f"  [ERROR] {e}")
        return None


# ============================================================================
# 2.5: PROCESS ALL DATASETS
# ============================================================================

def process_all_datasets():
    """Format and save all datasets"""

    results = {}

    for task_key, task_config in DATASET_CONFIG.items():
        print(f"\n{'='*70}")
        print(f"TASK: {task_config['task_name'].upper()}")
        print(f"{'='*70}")

        task_dir = Path(f"data/formatted/{task_key}")
        task_dir.mkdir(parents=True, exist_ok=True)

        individual_datasets = {}
        all_splits_for_merge = {"train": [], "valid": [], "test": []}

        for dataset_key, dataset_info in task_config['datasets'].items():
            formatted = format_dataset(
                task_key=task_key,
                dataset_key=dataset_key,
                hf_path=dataset_info['hf_path'],
                config=task_config,
                dataset_info=dataset_info
            )

            if formatted:
                individual_path = task_dir / dataset_key
                formatted.save_to_disk(str(individual_path))
                print(f"  [OK] Saved: {individual_path}")

                individual_datasets[dataset_key] = {
                    "path": str(individual_path),
                    "sizes": {split: len(formatted[split]) for split in formatted.keys()}
                }

                for split in ['train', 'valid', 'test']:
                    if split in formatted:
                        all_splits_for_merge[split].append(formatted[split])
                    elif split == 'valid' and 'validation' in formatted:
                        all_splits_for_merge[split].append(formatted['validation'])

        if any(all_splits_for_merge.values()):
            print(f"\n  Merging {len(task_config['datasets'])} dataset(s)...")

            merged = {}
            for split, datasets in all_splits_for_merge.items():
                if datasets:
                    merged[split] = concatenate_datasets(datasets)
                    print(f"    {split}: {len(merged[split]):,} samples")

            merged_dataset = DatasetDict(merged)
            merged_path = task_dir / "merged"
            merged_dataset.save_to_disk(str(merged_path))
            print(f"  [OK] Merged saved: {merged_path}")

            results[task_key] = {
                "task_name": task_config['task_name'],
                "merged_path": str(merged_path),
                "merged_sizes": {split: len(merged[split]) for split in merged.keys()},
                "individual_datasets": individual_datasets
            }

    print(f"\n{'='*70}")
    print("SUMMARY")
    print(f"{'='*70}\n")

    for task_key, task_results in results.items():
        print(f"{task_key.upper()}:")
        print(f"  Merged: {task_results['merged_sizes']}")
        print(f"  Individual: {len(task_results['individual_datasets'])}")
        for ds_name, ds_info in task_results['individual_datasets'].items():
            print(f"    - {ds_name}: {ds_info['sizes']}")
        print()

    with open("data/formatted/metadata.json", "w") as f:
        json.dump(results, f, indent=2)

    print("[OK] Metadata saved: data/formatted/metadata.json")
    print("="*70 + "\n")

    return results


# ============================================================================
# RUN
# ============================================================================

if __name__ == "__main__":
    print("="*80)
    print("DATASET FORMATTING PIPELINE")
    print("="*80 + "\n")

    random.seed(42)
    results = process_all_datasets()

    print("\n" + "="*80)
    print("[OK] FORMATTING COMPLETE")
    print("="*80 + "\n")
    print("Next steps:")
    print("  1. Verify: python scripts/verify_datasets.py")
    print("  2. Train: python scripts/train.py --task sa")
    print("  3. Evaluate: python scripts/evaluate.py --task sa")

STEP 2: DATASET FORMATTING

[OK] Configuration loaded

DATASET FORMATTING PIPELINE


TASK: SENTIMENT ANALYSIS

  Loading fpb from ChanceFocus/en-fpb...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/742 [00:00<?, ?B/s]

data/train-00000-of-00001-ab9a3b4799b095(…):   0%|          | 0.00/608k [00:00<?, ?B/s]

data/test-00000-of-00001-8bd1e21c671fb67(…):   0%|          | 0.00/188k [00:00<?, ?B/s]

...

  [OK] Saved: data/formatted/smp/stock_bigdata

  Merging 2 dataset(s)...
    train: 8,293 samples
    valid: 1,229 samples
    test: 2,615 samples


Saving the dataset (0/1 shards):   0%|          | 0/8293 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1229 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2615 [00:00<?, ? examples/s]

  [OK] Merged saved: data/formatted/smp/merged

SUMMARY

SA:
  Merged: {'train': 3849, 'valid': 964, 'test': 1205}
  Individual: 2
    - fpb: {'train': 3099, 'test': 970, 'valid': 776}
    - fiqasa: {'train': 750, 'test': 235, 'valid': 188}

HC:
  Merged: {'train': 71883, 'valid': 10269, 'test': 20547}
  Individual: 1
    - headlines: {'train': 71883, 'test': 20547, 'valid': 10269}

NER:
  Merged: {'train': 398, 'valid': 98, 'test': 98}
  Individual: 1
    - flare_ner: {'train': 398, 'test': 98, 'valid': 98}

QA:
  Merged: {'train': 15142, 'valid': 3096, 'test': 2637}
  Individual: 2
    - finqa: {'train': 6251, 'test': 1147, 'valid': 883}
    - convfinqa: {'train': 8891, 'test': 1490, 'valid': 2213}

SMP:
  Merged: {'train': 8293, 'valid': 1229, 'test': 2615}
  Individual: 2
    - stock_cikm: {'train': 3396, 'test': 1143, 'valid': 431}
    - stock_bigdata: {'train': 4897, 'validation': 798, 'test': 1472}

[OK] Metadata saved: data/formatted/metadata.json


[OK] FORMATTING COMPLETE

Ne

In [ ]:
"""
Inspect NER Dataset - Compare BEFORE and AFTER cleaning
"""

from datasets import load_dataset
from collections import OrderedDict

def clean_ner_answer(answer_text):
    if not answer_text:
        return None

    lines = answer_text.strip().split('\n')
    entities_dict = OrderedDict()

    for line in lines:
        line = line.strip()
        if not line or ',' not in line:
            continue
        parts = line.rsplit(',', 1)
        if len(parts) != 2:
            continue
        entity_name = parts[0].strip()
        entity_type = parts[1].strip().upper()
        if entity_type not in ['PER', 'ORG', 'LOC']:
            continue
        if not entity_name:
            continue
        key = (entity_name.lower(), entity_type)
        if key not in entities_dict:
            entities_dict[key] = entity_name

    if not entities_dict:
        return None

    cleaned_lines = [
        f"{entity_name}, {entity_type}"
        for (entity_name_lower, entity_type), entity_name in entities_dict.items()
    ]

    return '\n'.join(cleaned_lines)


print("="*80)
print("LOADING NER DATASET FROM HUGGING FACE")
print("="*80 + "\n")

dataset = load_dataset("TheFinAI/flare-ner")

print(f"Splits disponibles: {list(dataset.keys())}\n")


def inspect_split(split_name, num_samples=10):
    print("="*80)
    print(f"SPLIT: {split_name.upper()} - {num_samples} premiers exemples")
    print("="*80 + "\n")

    split_data = dataset[split_name]

    total_duplicates = 0
    total_before = 0
    total_after = 0

    for i in range(min(num_samples, len(split_data))):
        example = split_data[i]
        answer_before = example.get('answer', '')

        if not answer_before:
            print(f"Exemple {i+1}: PAS DE RÉPONSE\n")
            continue

        answer_after = clean_ner_answer(answer_before)

        lines_before = [l.strip() for l in answer_before.split('\n') if l.strip()]
        lines_after = [l.strip() for l in answer_after.split('\n') if l.strip()] if answer_after else []

        count_before = len(lines_before)
        count_after = len(lines_after)
        duplicates = count_before - count_after

        total_before += count_before
        total_after += count_after
        total_duplicates += duplicates

        print(f"{'─'*80}")
        print(f"EXEMPLE {i+1}")
        print(f"{'─'*80}")
        print(f"Entités AVANT: {count_before} | APRÈS: {count_after} | Doublons: {duplicates}")

        print(f"\n📝 AVANT nettoyage:")
        print(answer_before)

        print(f"\n✨ APRÈS nettoyage:")
        print(answer_after if answer_after else "VIDE")
        print()

    print("="*80)
    print(f"STATISTIQUES {split_name.upper()}")
    print("="*80)
    print(f"Total entités AVANT: {total_before}")
    print(f"Total entités APRÈS: {total_after}")
    print(f"Total doublons supprimés: {total_duplicates}")
    if total_before > 0:
        reduction = (total_duplicates / total_before) * 100
        print(f"Réduction: {reduction:.1f}%")
    print("\n")


# Inspect each split
for split in ['train', 'test']:
    if split in dataset:
        inspect_split(split, num_samples=10)

if 'valid' in dataset:
    inspect_split('valid', num_samples=10)
elif 'validation' in dataset:
    inspect_split('validation', num_samples=10)


print("="*80)
print(" INSPECTION TERMINÉE")
print("="*80)

LOADING NER DATASET FROM HUGGING FACE

Splits disponibles: ['train', 'test', 'valid']

SPLIT: TRAIN - 10 premiers exemples

────────────────────────────────────────────────────────────────────────────────
EXEMPLE 1
────────────────────────────────────────────────────────────────────────────────
Entités AVANT: 8 | APRÈS: 6 | Doublons: 2

📝 AVANT nettoyage:
SILICON VALLEY BANK, ORG
Bank, ORG
California, LOC
bank, ORG
3003 Tasman Drive, LOC
Santa Clara, LOC
California, LOC
40 William St, LOC

✨ APRÈS nettoyage:
SILICON VALLEY BANK, ORG
Bank, ORG
California, LOC
3003 Tasman Drive, LOC
Santa Clara, LOC
40 William St, LOC

────────────────────────────────────────────────────────────────────────────────
EXEMPLE 2
────────────────────────────────────────────────────────────────────────────────
Entités AVANT: 12 | APRÈS: 8 | Doublons: 4

📝 AVANT nettoyage:
Wellesley, LOC
Massachusetts, LOC
Silicon Valley East, LOC
AKAMAI TECHNOLOGIES, ORG
Borrower, PER
201 Broadway, LOC
Cambridge, LOC
Massachus

# ***CELL 3: MODEL & TRAINING CONFIGURATIONS***

In [ ]:
"""
Crée les fichiers de configuration pour le modèle et l'entraînement
"""

print("="*80)
print("STEP 3: CREATING CONFIGURATIONS")
print("="*80 + "\n")

import yaml
from pathlib import Path

# ============================================================================
# 3.1: MODEL CONFIGURATION
# ============================================================================

model_config = {
    "model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct",

    "quantization": {
        "load_in_4bit": True,
        "bnb_4bit_quant_type": "nf4",
        "bnb_4bit_compute_dtype": "bfloat16",
        "bnb_4bit_use_double_quant": True
    },

    "lora_common": {
        "lora_dropout": 0.05,
        "bias": "none",
        "task_type": "CAUSAL_LM",
        "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"]
    }
}

with open("configs/model_config.yaml", "w") as f:
    yaml.dump(model_config, f, default_flow_style=False)

print("[OK] configs/model_config.yaml created\n")


# ============================================================================
# 3.2: TASK-SPECIFIC CONFIGURATIONS
# ============================================================================

task_configs = {
    "sa": {
        "task_name": "sentiment_analysis",
        "adapter_name": "sa_adapter",
        "dataset_path": "data/formatted/sa/merged",
        "max_sequence_length": 512,
        "lora": {
            "r": 8,
            "lora_alpha": 16
        },
        "training_args": {
            "num_epochs": 3,
            "learning_rate": 0.0002,
            "per_device_train_batch_size": 16,
            "gradient_accumulation_steps": 8,
            "weight_decay": 0.01,
            "warmup_ratio": 0.03,
            "max_grad_norm": 1.0,
            "eval_strategy": "steps",
            "eval_steps": 100,
            "save_steps": 100,
            "logging_steps": 50,
            "load_best_model_at_end": True,
            "metric_for_best_model": "eval_loss",
            "greater_is_better": False
        }
    },

    "hc": {
        "task_name": "headline_classification",
        "adapter_name": "hc_adapter",
        "dataset_path": "data/formatted/hc/merged",
        "max_sequence_length": 128,
        "lora": {
            "r": 8,
            "lora_alpha": 16
        },
        "training_args": {
            "num_epochs": 1,
            "learning_rate": 1e-4,
            "per_device_train_batch_size": 8,
            "gradient_accumulation_steps": 4,
            "weight_decay": 0.01,
            "warmup_ratio": 0.03,
            "max_grad_norm": 1.0,
            "eval_strategy": "steps",
            "eval_steps": 500,
            "save_steps": 500,
            "logging_steps": 50,
            "load_best_model_at_end": True,
            "metric_for_best_model": "eval_loss",
            "greater_is_better": False
        }
    },

    "ner": {
        "task_name": "named_entity_recognition",
        "adapter_name": "ner_adapter",
        "dataset_path": "data/formatted/ner/merged",
        "max_sequence_length": 1024,
        "lora": {
            "r": 8,
            "lora_dropout": 0.15,
            "lora_alpha": 16,
        },
        "training_args": {
            "num_epochs": 2,
            "learning_rate": 3e-5,
            "per_device_train_batch_size": 8,
            "gradient_accumulation_steps": 4,
            "weight_decay": 0.05,
            "warmup_ratio": 0.1,
            "max_grad_norm": 1.0,
            "eval_strategy": "steps",
            "eval_steps": 25,
            "save_steps": 25,
            "logging_steps": 25,
            "load_best_model_at_end": True,
            "metric_for_best_model": "eval_loss",
            "greater_is_better": False
        }
    },

    "qa": {
        "task_name": "question_answering",
        "adapter_name": "qa_adapter",
        "dataset_path": "data/formatted/qa/merged",
        "max_sequence_length": 2048,
        "lora": {
            "r": 16,
            "lora_alpha": 32
        },
        "training_args": {
            "num_epochs": 2,
            "learning_rate": 0.0002,
            "per_device_train_batch_size": 8,
            "gradient_accumulation_steps": 2,
            "weight_decay": 0.01,
            "warmup_ratio": 0.03,
            "max_grad_norm": 1.0,
            "eval_strategy": "steps",
            "eval_steps": 200,
            "save_steps": 200,
            "logging_steps": 50,
            "load_best_model_at_end": True,
            "metric_for_best_model": "eval_loss",
            "greater_is_better": False
        }
    },

    "smp": {
        "task_name": "stock_movement_prediction",
        "adapter_name": "smp_adapter",
        "dataset_path": "data/formatted/smp/merged",
        "max_sequence_length": 2048,
        "lora": {
            "r": 32,
            "lora_alpha": 64
        },
        "training_args": {
            "num_epochs": 2,
            "learning_rate": 2e-4,
            "per_device_train_batch_size": 8,
            "gradient_accumulation_steps": 2,
            "weight_decay": 0.01,
            "warmup_ratio": 0.05,
            "max_grad_norm": 1.0,
            "eval_strategy": "steps",
            "eval_steps": 250,
            "save_steps": 250,
            "logging_steps": 50,
            "load_best_model_at_end": True,
            "metric_for_best_model": "eval_loss",
            "greater_is_better": False
        }
    }
}

print("📝 Creating task configs:\n")
for task_key, config in task_configs.items():
    config_path = f"configs/tasks/{task_key}_config.yaml"
    with open(config_path, "w") as f:
        yaml.dump(config, f, default_flow_style=False)
    print(f"  [OK] {config_path}")
    print(f"     Dataset: {config['dataset_path']}")
    print(f"     Epochs: {config['training_args']['num_epochs']}")
    print(f"     LR: {config['training_args']['learning_rate']}")
    print()

print("="*80)
print("[OK] ALL CONFIGURATIONS CREATED")
print("="*80 + "\n")

STEP 3: CREATING CONFIGURATIONS

[OK] configs/model_config.yaml created

📝 Creating task configs:

  [OK] configs/tasks/sa_config.yaml
     Dataset: data/formatted/sa/merged
     Epochs: 3
     LR: 0.0002

  [OK] configs/tasks/hc_config.yaml
     Dataset: data/formatted/hc/merged
     Epochs: 1
     LR: 0.0001

  [OK] configs/tasks/ner_config.yaml
     Dataset: data/formatted/ner/merged
     Epochs: 2
     LR: 3e-05

  [OK] configs/tasks/qa_config.yaml
     Dataset: data/formatted/qa/merged
     Epochs: 2
     LR: 0.0002

  [OK] configs/tasks/smp_config.yaml
     Dataset: data/formatted/smp/merged
     Epochs: 2
     LR: 0.0002

[OK] ALL CONFIGURATIONS CREATED



# ***CELL 4: CORE COMPONENTS***


In [ ]:
"""
Crée les composants principaux: Model, Loader, Utils
"""

print("="*80)
print("STEP 4: CREATING CORE COMPONENTS")
print("="*80 + "\n")

# ============================================================================
# 4.1: DATASET LOADER
# ============================================================================

dataset_loader_code = """\"\"\"
Dataset Loader for Training
Charge les datasets mergés pour l'entraînement
\"\"\"

from datasets import load_from_disk, DatasetDict
from pathlib import Path
import json
import re


class DatasetLoader:
    \"\"\"Load merged datasets for training\"\"\"

    def __init__(self):
        # Load metadata
        with open("data/formatted/metadata.json", "r") as f:
            self.metadata = json.load(f)

    def load_task_dataset(self, task_key: str) -> DatasetDict:
        \"\"\"
        Load merged dataset for a specific task

        Args:
            task_key: Task identifier (sa, hc, ner, qa, smp)

        Returns:
            DatasetDict with train/valid/test splits
        \"\"\"
        if task_key not in self.metadata:
            available = list(self.metadata.keys())
            raise ValueError(f"Unknown task: {task_key}. Available: {available}")

        task_info = self.metadata[task_key]
        merged_path = Path(task_info['merged_path'])

        if not merged_path.exists():
            raise FileNotFoundError(
                f"Dataset not found: {merged_path}\\n"
                f"Run: python scripts/1_format_datasets.py"
            )

        # Load dataset
        dataset = load_from_disk(str(merged_path))

        # Print info
        print(f"[OK] Loaded: {task_key.upper()}")
        print(f"     Path: {merged_path}")
        print(f"     Splits: {list(dataset.keys())}")
        for split in dataset.keys():
            print(f"     {split}: {len(dataset[split]):,} samples")

        return dataset

    def validate_format(self, dataset: DatasetDict, task_key: str) -> bool:
        \"\"\"Validate Llama 3.1 format with task-specific label validation\"\"\"
        print(f"\\n{'='*60}")
        print(f"VALIDATING: {task_key.upper()}")
        print(f"{'='*60}")

        if 'train' not in dataset or len(dataset['train']) == 0:
            print("[ERROR] No training data found")
            return False

        # Get task config for labels and max_length
        with open("data/dataset_config.json", "r") as f:
            task_config = json.load(f)[task_key]

        expected_labels = task_config.get('labels')
        max_length = task_config.get('max_length', 2048)

        # Check multiple samples
        print("\\nChecking 5 random samples...")
        import random
        indices = random.sample(range(len(dataset['train'])), min(5, len(dataset['train'])))

        issues = []
        valid_labels_found = []

        for idx in indices:
            sample = dataset['train'][idx]['text']

            # 1. Check structure
            required_tokens = [
                '<|begin_of_text|>',
                '<|start_header_id|>system<|end_header_id|>',
                '<|start_header_id|>user<|end_header_id|>',
                '<|start_header_id|>assistant<|end_header_id|>',
                '<|eot_id|>'
            ]

            missing = [t for t in required_tokens if t not in sample]
            if missing:
                issues.append(f"Sample {idx}: Missing tokens {missing}")
                continue

            # 2. Check user text integrity
            if "<|start_header_id|>user<|end_header_id|>" in sample:
                user_part = sample.split("<|start_header_id|>user<|end_header_id|>")[1]
                user_text = user_part.split("<|start_header_id|>assistant<|end_header_id|>")[0].strip()

                if "Answer:" in user_text:
                    issues.append(f"Sample {idx}: User text contains 'Answer:' (potential data leakage)")

                if not user_text or len(user_text) < 10:
                    issues.append(f"Sample {idx}: User text too short or empty ({len(user_text)} chars)")

            # 3. Extract and validate assistant response
            if "<|start_header_id|>assistant<|end_header_id|>" in sample:
                assistant_part = sample.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
                assistant_response = assistant_part.split("<|eot_id|>")[0].strip()

                if not assistant_response:
                    issues.append(f"Sample {idx}: Empty assistant response")
                    continue

                # 4. Task-specific label validation
                if expected_labels:
                    if task_key == "ner":
                        # NER: Extract entity types (last element after comma)
                        lines = assistant_response.split('\\n')
                        for line in lines:
                            if ',' in line:
                                parts = line.rsplit(',', 1)
                                if len(parts) == 2:
                                    entity_type = parts[1].strip()
                                    if entity_type not in expected_labels:
                                        issues.append(f"Sample {idx}: Invalid NER type '{entity_type}' (expected: {expected_labels})")
                                    else:
                                        valid_labels_found.append(entity_type)
                    else:
                        # Classification: First line is the label
                        label = assistant_response.split('\\n')[0].strip()
                        if label not in expected_labels:
                            issues.append(f"Sample {idx}: Invalid label '{label}' (expected: {expected_labels})")
                        else:
                            valid_labels_found.append(label)

            # 5. Check estimated token count
            estimated_tokens = len(sample) // 4
            if estimated_tokens > max_length:
                issues.append(f"Sample {idx}: Estimated {estimated_tokens} tokens exceeds max {max_length}")

        # Display label distribution
        if valid_labels_found:
            from collections import Counter
            counts = Counter(valid_labels_found)
            print(f"\\n  Label distribution in validated samples:")
            for label, count in sorted(counts.items()):
                print(f"    {label}: {count}/{len(valid_labels_found)}")

        if issues:
            print(f"\\n[ERROR] VALIDATION FAILED:")
            for issue in issues[:10]:
                print(f"  - {issue}")
            if len(issues) > 10:
                print(f"  ... and {len(issues) - 10} more issues")
            return False

        print("[OK] Format validation passed")
        print(f"{'='*60}\\n")
        return True

    def get_task_info(self, task_key: str) -> dict:
        \"\"\"Get metadata for a task\"\"\"
        return self.metadata.get(task_key)

    def list_tasks(self):
        \"\"\"List all available tasks\"\"\"
        print("\\nAvailable tasks:")
        for task_key, info in self.metadata.items():
            print(f"  - {task_key}: {info['task_name']}")
            print(f"    Merged samples: {info['merged_sizes']}")
            print(f"    Individual datasets: {len(info['individual_datasets'])}")
"""

with open("src/data/dataset_loader.py", "w") as f:
    f.write(dataset_loader_code)

print("[OK] src/data/dataset_loader.py created")


# ============================================================================
# 4.2: QLORA MODEL
# ============================================================================

qlora_model_code = """\"\"\"
QLoRA Model Wrapper
Gère le chargement du modèle avec quantization et LoRA
\"\"\"

import torch
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import yaml


class QLoRAModel:
    \"\"\"QLoRA Model Manager\"\"\"

    def __init__(self, model_config_path: str = "configs/model_config.yaml"):
        with open(model_config_path, "r") as f:
            self.config = yaml.safe_load(f)

        self.model_name = self.config["model_name"]
        self.model = None
        self.tokenizer = None

    def create_quantization_config(self):
        \"\"\"Create BitsAndBytes config for 4-bit quantization\"\"\"
        qconfig = self.config["quantization"]

        return BitsAndBytesConfig(
            load_in_4bit=qconfig["load_in_4bit"],
            bnb_4bit_quant_type=qconfig["bnb_4bit_quant_type"],
            bnb_4bit_compute_dtype=getattr(torch, qconfig["bnb_4bit_compute_dtype"]),
            bnb_4bit_use_double_quant=qconfig["bnb_4bit_use_double_quant"]
        )

    def create_lora_config(self, task_config: dict):
        \"\"\"Create LoRA config for a specific task\"\"\"
        lora_params = task_config["lora"]
        lora_common = self.config["lora_common"]

        return LoraConfig(
            r=lora_params["r"],
            lora_alpha=lora_params["lora_alpha"],
            target_modules=lora_common["target_modules"],
            lora_dropout=lora_params.get("lora_dropout", lora_common["lora_dropout"]),
            bias=lora_common["bias"],
            task_type=TaskType.CAUSAL_LM
        )

    def load_model(self, task_config: dict):
        \"\"\"Load model with quantization and LoRA\"\"\"
        print(f"\\n[INFO] Loading model: {self.model_name}")

        # Quantization config
        bnb_config = self.create_quantization_config()

        # Load base model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.bfloat16,
        )

        # Prepare for training
        self.model.gradient_checkpointing_enable()
        self.model = prepare_model_for_kbit_training(self.model)
        self.model.config.use_cache = False

        # Add LoRA adapters
        lora_config = self.create_lora_config(task_config)
        self.model = get_peft_model(self.model, lora_config)

        print("[OK] Model loaded successfully")
        self.model.print_trainable_parameters()

        return self.model

    def load_tokenizer(self):
        \"\"\"Load tokenizer\"\"\"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        print(f"[OK] Tokenizer loaded")
        return self.tokenizer
"""

with open("src/models/qlora_model.py", "w") as f:
    f.write(qlora_model_code)

print("[OK] src/models/qlora_model.py created")


# ============================================================================
# 4.3: TRAINING MONITOR (VERSION AMÉLIORÉE POUR ARTICLE SCIENTIFIQUE)
# ============================================================================

training_monitor_code = """\"\"\"
Training Monitor
Suit les métriques et génère des visualisations pour article scientifique
\"\"\"

import time
import json
import matplotlib.pyplot as plt
import torch
import numpy as np
from pathlib import Path
from datetime import timedelta


class TrainingMonitor:
    \"\"\"Monitor training progress and metrics\"\"\"

    def __init__(self, output_dir: str):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)

        self.metrics = {
            'train_loss': [],
            'eval_loss': [],
            'learning_rate': [],
            'grad_norm': [],
            'steps': [],
            'eval_steps': [],
            'timestamps': [],
            'gpu_memory_current': [],  # Mémoire baseline actuelle
            'gpu_memory_peak': []       # Peak entre chaque log
        }

        self.start_time = time.time()

        # Reset peak memory stats au début du training
        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

    def update(self, step: int, metrics: dict):
        \"\"\"Update metrics\"\"\"
        self.metrics['steps'].append(step)
        self.metrics['timestamps'].append(time.time() - self.start_time)

        if 'loss' in metrics:
            self.metrics['train_loss'].append(float(metrics['loss']))

        if 'learning_rate' in metrics:
            self.metrics['learning_rate'].append(float(metrics['learning_rate']))

        if 'grad_norm' in metrics:
            self.metrics['grad_norm'].append(float(metrics['grad_norm']))

        if 'eval_loss' in metrics:
            self.metrics['eval_loss'].append(float(metrics['eval_loss']))
            self.metrics['eval_steps'].append(step)

        # GPU Memory tracking avec current + peak
        if torch.cuda.is_available():
            # Current allocated memory (baseline)
            current_gb = torch.cuda.memory_allocated() / (1024**3)
            self.metrics['gpu_memory_current'].append(current_gb)

            # Peak memory since last update (captures spikes)
            peak_gb = torch.cuda.max_memory_allocated() / (1024**3)
            self.metrics['gpu_memory_peak'].append(peak_gb)

            # Reset peak counter pour mesurer le prochain intervalle
            torch.cuda.reset_peak_memory_stats()

    def print_summary(self):
        \"\"\"Print training summary\"\"\"
        print("\\n" + "="*70)
        print("TRAINING SUMMARY")
        print("="*70)

        if self.metrics['train_loss']:
            start = self.metrics['train_loss'][0]
            end = self.metrics['train_loss'][-1]
            reduction = ((start - end) / start) * 100
            print(f"Train Loss: {start:.4f} -> {end:.4f} ({reduction:.1f}% reduction)")

        if self.metrics['eval_loss']:
            start = self.metrics['eval_loss'][0]
            end = self.metrics['eval_loss'][-1]
            reduction = ((start - end) / start) * 100
            print(f"Eval Loss:  {start:.4f} -> {end:.4f} ({reduction:.1f}% reduction)")

        if self.metrics['timestamps']:
            total = self.metrics['timestamps'][-1]
            print(f"Time: {timedelta(seconds=int(total))}")

        # GPU Memory statistics (pour article scientifique)
        if self.metrics['gpu_memory_peak']:
            peak_max = max(self.metrics['gpu_memory_peak'])
            current_avg = np.mean(self.metrics['gpu_memory_current'])
            current_std = np.std(self.metrics['gpu_memory_current'])
            overhead = ((peak_max - current_avg) / current_avg) * 100

            print(f"\\nGPU Memory:")
            print(f"  Baseline (avg ± std): {current_avg:.2f} ± {current_std:.2f} GB")
            print(f"  Peak (max):           {peak_max:.2f} GB")
            print(f"  Memory Overhead:      {overhead:.1f}%")

        print("="*70 + "\\n")

    def save_metrics(self):
        \"\"\"Save metrics to JSON\"\"\"
        metrics_file = self.output_dir / 'metrics.json'

        # Calculate additional statistics for paper
        stats = {}
        if self.metrics['gpu_memory_peak']:
            stats['gpu_baseline_mean'] = float(np.mean(self.metrics['gpu_memory_current']))
            stats['gpu_baseline_std'] = float(np.std(self.metrics['gpu_memory_current']))
            stats['gpu_peak_max'] = float(max(self.metrics['gpu_memory_peak']))
            stats['gpu_overhead_percent'] = float(
                ((stats['gpu_peak_max'] - stats['gpu_baseline_mean']) / stats['gpu_baseline_mean']) * 100
            )

        # Save metrics with stats
        output = {
            'metrics': self.metrics,
            'statistics': stats
        }

        with open(metrics_file, 'w') as f:
            json.dump(output, f, indent=2)

        print(f"[OK] Metrics saved: {metrics_file}")

        self.plot_metrics()

    def plot_metrics(self):
        \"\"\"Create training plots for scientific paper\"\"\"
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Training Dashboard', fontsize=16, fontweight='bold')
        axes = axes.flatten()

        # Loss
        ax = axes[0]
        if self.metrics['train_loss']:
            steps = self.metrics['steps'][:len(self.metrics['train_loss'])]
            ax.plot(steps, self.metrics['train_loss'], label='Train', linewidth=2, alpha=0.8)
        if self.metrics['eval_loss']:
            ax.plot(self.metrics['eval_steps'], self.metrics['eval_loss'],
                   label='Eval', marker='o', linewidth=2, alpha=0.8)
        ax.set_xlabel('Steps', fontsize=11)
        ax.set_ylabel('Loss', fontsize=11)
        ax.set_title('Loss Evolution', fontsize=12, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(alpha=0.3)

        # Learning Rate
        ax = axes[1]
        if self.metrics['learning_rate']:
            steps = self.metrics['steps'][:len(self.metrics['learning_rate'])]
            ax.plot(steps, self.metrics['learning_rate'], linewidth=2, color='#2ca02c')
        ax.set_xlabel('Steps', fontsize=11)
        ax.set_ylabel('Learning Rate', fontsize=11)
        ax.set_title('Learning Rate Schedule', fontsize=12, fontweight='bold')
        ax.ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
        ax.grid(alpha=0.3)

        # Gradient Norm
        ax = axes[2]
        if self.metrics['grad_norm']:
            steps = self.metrics['steps'][:len(self.metrics['grad_norm'])]
            ax.plot(steps, self.metrics['grad_norm'], linewidth=2, color='#d62728')
        ax.set_xlabel('Steps', fontsize=11)
        ax.set_ylabel('Gradient Norm', fontsize=11)
        ax.set_title('Gradient Norm', fontsize=12, fontweight='bold')
        ax.grid(alpha=0.3)

        # GPU Memory - ENHANCED avec current + peak + overhead zone
        ax = axes[3]
        if self.metrics['gpu_memory_current'] and self.metrics['gpu_memory_peak']:
            steps = self.metrics['steps'][:len(self.metrics['gpu_memory_current'])]

            # Current memory (baseline)
            ax.plot(steps, self.metrics['gpu_memory_current'],
                    label='Current (Baseline)', linewidth=2, alpha=0.7, color='#1f77b4')

            # Peak memory
            ax.plot(steps, self.metrics['gpu_memory_peak'],
                    label='Peak', linewidth=2, linestyle='--', alpha=0.7, color='#ff7f0e')

            # Fill area between current and peak (overhead)
            ax.fill_between(steps,
                            self.metrics['gpu_memory_current'],
                            self.metrics['gpu_memory_peak'],
                            alpha=0.2, color='#ff7f0e', label='Gradient Overhead')

            # Add statistics annotation
            peak_max = max(self.metrics['gpu_memory_peak'])
            current_avg = np.mean(self.metrics['gpu_memory_current'])
            overhead_pct = ((peak_max - current_avg) / current_avg) * 100

            ax.text(0.02, 0.98,
                   f'Baseline: {current_avg:.1f} GB\\nPeak: {peak_max:.1f} GB\\nOverhead: {overhead_pct:.0f}%',
                   transform=ax.transAxes, fontsize=9,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        ax.set_xlabel('Steps', fontsize=11)
        ax.set_ylabel('Memory (GB)', fontsize=11)
        ax.set_title('GPU Memory Usage', fontsize=12, fontweight='bold')
        ax.legend(fontsize=9, loc='upper left')
        ax.grid(alpha=0.3)

        plt.tight_layout()
        plot_path = self.output_dir / 'training_metrics.png'
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')  # High DPI pour article
        plt.close()

        print(f"[OK] Plot saved: {plot_path}")
"""

with open("src/utils/training_monitor.py", "w") as f:
    f.write(training_monitor_code)

print("[OK] src/utils/training_monitor.py created (ENHANCED for scientific paper)")


# ============================================================================
# 4.4: MONITORING CALLBACK
# ============================================================================

callback_code = """\"\"\"
Training Callback
Intègre le monitor dans le trainer
\"\"\"

from transformers import TrainerCallback
from src.utils.training_monitor import TrainingMonitor


class MonitoringCallback(TrainerCallback):
    \"\"\"Callback to monitor training\"\"\"

    def __init__(self, monitor: TrainingMonitor):
        self.monitor = monitor

    def on_log(self, args, state, control, logs=None, **kwargs):
        \"\"\"Called when logging\"\"\"
        if logs and state.global_step > 0:
            self.monitor.update(state.global_step, logs)

    def on_train_begin(self, args, state, control, **kwargs):
        \"\"\"Called at training start\"\"\"
        print("\\n" + "="*70)
        print("TRAINING STARTED")
        print("="*70)
        print(f"Total steps: {state.max_steps}")
        print(f"Eval every: {args.eval_steps} steps")
        print("="*70 + "\\n")

    def on_train_end(self, args, state, control, **kwargs):
        \"\"\"Called at training end\"\"\"
        print("\\n" + "="*70)
        print("TRAINING COMPLETED")
        print("="*70 + "\\n")

        self.monitor.print_summary()
        self.monitor.save_metrics()
"""

with open("src/training/callbacks.py", "w") as f:
    f.write(callback_code)

print("[OK] src/training/callbacks.py created")

print("\n" + "="*80)
print("[OK] ALL CORE COMPONENTS CREATED")
print("="*80 + "\n")

STEP 4: CREATING CORE COMPONENTS

[OK] src/data/dataset_loader.py created
[OK] src/models/qlora_model.py created
[OK] src/utils/training_monitor.py created (ENHANCED for scientific paper)
[OK] src/training/callbacks.py created

[OK] ALL CORE COMPONENTS CREATED



# ***CELL 5: TRAINER***


In [ ]:
"""
Crée le trainer principal pour l'entraînement
"""

print("="*80)
print("STEP 5: CREATING TRAINER")
print("="*80 + "\n")

trainer_code = """\"\"\"
Task Trainer
Gère l'entraînement d'un adaptateur pour une tâche spécifique
\"\"\"

import torch
import yaml
from pathlib import Path
from transformers import EarlyStoppingCallback
from trl import SFTTrainer, SFTConfig

from src.models.qlora_model import QLoRAModel
from src.data.dataset_loader import DatasetLoader
from src.utils.training_monitor import TrainingMonitor
from src.training.callbacks import MonitoringCallback


class TaskTrainer:
    \"\"\"Train QLoRA adapter for a specific task\"\"\"

    def __init__(self, task_key: str):
        self.task_key = task_key

        # Load task config
        config_path = f"configs/tasks/{task_key}_config.yaml"
        with open(config_path, "r") as f:
            self.task_config = yaml.safe_load(f)

        # Initialize components
        self.qlora_model = QLoRAModel()
        self.data_loader = DatasetLoader()

    def create_training_config(self, output_dir: str):
        \"\"\"Create SFTConfig for training\"\"\"
        tc = self.task_config["training_args"]

        return SFTConfig(
            output_dir=output_dir,
            overwrite_output_dir=True,

            # Training
            num_train_epochs=tc["num_epochs"],
            per_device_train_batch_size=tc["per_device_train_batch_size"],
            per_device_eval_batch_size=tc.get("per_device_eval_batch_size", 4),
            gradient_accumulation_steps=tc["gradient_accumulation_steps"],

            # Optimizer
            learning_rate=float(tc["learning_rate"]),
            weight_decay=float(tc["weight_decay"]),
            optim="paged_adamw_8bit",
            max_grad_norm=float(tc["max_grad_norm"]),

            # Scheduler
            lr_scheduler_type="cosine_with_restarts",
            warmup_ratio=float(tc["warmup_ratio"]),
            warmup_steps=100,

            # Logging
            logging_steps=tc["logging_steps"],

            # Saving
            save_strategy="steps",
            save_steps=tc["save_steps"],
            save_total_limit=3,

            # Evaluation
            eval_strategy=tc["eval_strategy"],
            eval_steps=tc["eval_steps"],

            # Best model selection
            load_best_model_at_end=tc["load_best_model_at_end"],
            metric_for_best_model=tc["metric_for_best_model"],
            greater_is_better=tc["greater_is_better"],

            # Precision
            fp16=False,
            bf16=True,

            # Data
            dataloader_pin_memory=True,
            remove_unused_columns=False,
            report_to="tensorboard",
            do_eval=True,

            dataset_text_field="text",
            max_length=self.task_config["max_sequence_length"],
        )

    def train(self):
        \"\"\"Train the model\"\"\"
        print(f"\\n{'='*70}")
        print(f"TRAINING TASK: {self.task_key.upper()}")
        print(f"{'='*70}\\n")

        # Load dataset
        print("[1/5] Loading dataset...")
        dataset = self.data_loader.load_task_dataset(self.task_key)

        # Validate format
        print("\\n[2/5] Validating format...")
        if not self.data_loader.validate_format(dataset, self.task_key):
            raise ValueError("Dataset format validation failed!")

        # Load tokenizer early for validation
        print("\\n[2.5/5] Loading tokenizer for validation...")
        tokenizer = self.qlora_model.load_tokenizer()

        # Check REAL tokenization on samples
        print("\\n[2.6/5] Validating tokenization...")
        import random
        sample_indices = random.sample(range(len(dataset['train'])), min(10, len(dataset['train'])))

        max_len = self.task_config['max_sequence_length']
        over_limit = 0

        for idx in sample_indices:
            sample_text = dataset['train'][idx]['text']
            tokens = tokenizer.encode(sample_text)

            if len(tokens) > max_len:
                over_limit += 1
                if over_limit == 1:  # Print first occurrence
                    print(f"  Sample {idx}: {len(tokens)} tokens > max {max_len}")

        if over_limit > 0:
            print(f"  WARNING: {over_limit}/{len(sample_indices)} samples exceed max length!")
            print(f"     SFTTrainer will automatically truncate during training")
        else:
            print(f"  [OK] All samples within token limit")

        # Load model and tokenizer
        print("\\n[3/5] Loading model and tokenizer...")
        model = self.qlora_model.load_model(self.task_config)
        tokenizer = self.qlora_model.load_tokenizer()

        # Ensure pad token is set
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = tokenizer.eos_token_id

        # Setup output directory
        output_dir = f"outputs/adapters/{self.task_config['adapter_name']}"
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        # Create training config
        print("\\n[4/5] Creating training configuration...")
        sft_config = self.create_training_config(output_dir)

        print(f"\\n  Configuration:")
        print(f"    Max length: {sft_config.max_length}")
        print(f"    Batch size: {sft_config.per_device_train_batch_size}")
        print(f"    Gradient accum: {sft_config.gradient_accumulation_steps}")
        print(f"    Learning rate: {sft_config.learning_rate}")
        print(f"    Epochs: {sft_config.num_train_epochs}")
        print(f"    Eval steps: {sft_config.eval_steps}")

        # Setup monitoring
        monitor = TrainingMonitor(output_dir)
        callback = MonitoringCallback(monitor)

        # Get eval dataset
        eval_dataset = dataset.get("valid", dataset.get("test"))

        # Create trainer
        print("\\n[5/5] Creating trainer...")
        trainer = SFTTrainer(
            model=model,
            args=sft_config,
            train_dataset=dataset["train"],
            eval_dataset=eval_dataset,
            processing_class=tokenizer,
            compute_metrics=None,  # Disabled to avoid OOM
            callbacks=[callback],
        )

        # Add early stopping
        if sft_config.load_best_model_at_end:
            trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
            print("  Early stopping enabled (patience=3)")

        # Train
        print(f"\\n{'='*70}")
        print("STARTING TRAINING")
        print(f"{'='*70}\\n")

        try:
            # Check if a checkpoint exists to resume training
            resume_from_checkpoint = None
            if any(Path(output_dir).glob("checkpoint-*")):
                checkpoints = list(Path(output_dir).glob("checkpoint-*"))

                # Helper function to safely extract checkpoint number
                def get_checkpoint_number(path):
                    try:
                        return int(path.name.split('-')[1])
                    except (ValueError, IndexError):
                        # Handle non-numeric checkpoints like 'checkpoint-best'
                        return -1

                # Filter only numeric checkpoints
                numeric_checkpoints = [c for c in checkpoints if get_checkpoint_number(c) > 0]

                if numeric_checkpoints:
                    latest = max(numeric_checkpoints, key=get_checkpoint_number)
                    resume_from_checkpoint = str(latest)
                    print(f"[INFO] Resuming training from checkpoint: {resume_from_checkpoint}")
                else:
                    print("[WARN] Found checkpoints but none are numeric, starting fresh")

            # Start training (resume if applicable)
            train_result = trainer.train(resume_from_checkpoint=resume_from_checkpoint)

            print(f"\\n{'='*70}")
            print("[OK] TRAINING COMPLETED SUCCESSFULLY")
            print(f"{'='*70}\\n")

        except Exception as e:
            print(f"\\n{'='*70}")
            print("TRAINING FAILED")
            print(f"{'='*70}")
            print(f"Error: {e}")

            monitor.print_summary()
            monitor.save_metrics()
            raise

        # Save final adapter
        print("[INFO] Saving final adapter...")
        adapter_path = f"{output_dir}/final_adapter"
        model.save_pretrained(adapter_path)
        tokenizer.save_pretrained(adapter_path)

        print(f"[OK] Adapter saved: {adapter_path}")

        # Final summary
        monitor.print_summary()
        monitor.save_metrics()

        return model, tokenizer
"""

with open("src/training/trainer.py", "w") as f:
    f.write(trainer_code)

print("[OK] src/training/trainer.py created\n")

print("="*80)
print("[OK] TRAINER CREATED")
print("="*80 + "\n")

STEP 5: CREATING TRAINER

[OK] src/training/trainer.py created

[OK] TRAINER CREATED



# test until training

In [ ]:
"""
Pre-Training Validation Script
Tests all components before launching fine-tuning

Run in notebook cell or: python scripts/test_before_training.py
"""

import sys
import torch
import yaml
import json
from pathlib import Path
from collections import Counter
import re

# Ensure we're in project root
import os
if 'scripts' in os.getcwd():
    os.chdir('..')

print(f"Working directory: {os.getcwd()}")

# Import directly from files instead of module
exec(open('src/data/dataset_loader.py').read())
exec(open('src/models/qlora_model.py').read())


class PreTrainingValidator:
    """Comprehensive validation before training"""

    def __init__(self):
        self.loader = DatasetLoader()
        self.results = {}

    def print_header(self, text):
        print("\n" + "="*80)
        print(f" {text}")
        print("="*80 + "\n")

    def print_section(self, text):
        print(f"\n{'─'*80}")
        print(f"  {text}")
        print(f"{'─'*80}\n")

    # ========================================================================
    # TEST 1: DATASET INTEGRITY
    # ========================================================================

    def test_dataset_integrity(self, task_key):
        """Test 1: Validate dataset format and structure"""
        self.print_section(f"TEST 1: Dataset Integrity - {task_key.upper()}")

        try:
            # Load dataset
            dataset = self.loader.load_task_dataset(task_key)

            # Check splits
            required_splits = ['train', 'valid', 'test']
            available_splits = list(dataset.keys())

            # Accept 'validation' as 'valid'
            if 'validation' in available_splits and 'valid' not in available_splits:
                available_splits.append('valid')

            missing_splits = [s for s in required_splits if s not in available_splits]

            if missing_splits:
                print(f"  [WARNING] Missing splits: {missing_splits}")
            else:
                print(f"  [OK] All required splits present")

            # Check non-empty
            empty_splits = [s for s in dataset.keys() if len(dataset[s]) == 0]
            if empty_splits:
                print(f"  [ERROR] Empty splits: {empty_splits}")
                return False
            else:
                print(f"  [OK] All splits contain data")

            # Print sizes
            print("\n  Split sizes:")
            for split in dataset.keys():
                print(f"    {split}: {len(dataset[split]):,} samples")

            return True

        except Exception as e:
            print(f"  [ERROR] Failed to load dataset: {e}")
            import traceback
            traceback.print_exc()
            return False

    # ========================================================================
    # TEST 2: FORMAT VALIDATION
    # ========================================================================

    def test_format_validation(self, task_key):
        """Test 2: Validate Llama 3.1 chat format"""
        self.print_section(f"TEST 2: Format Validation - {task_key.upper()}")

        try:
            dataset = self.loader.load_task_dataset(task_key)

            # Load task config
            with open("data/dataset_config.json", "r") as f:
                task_config = json.load(f)[task_key]

            expected_labels = task_config.get('labels')

            # Sample validation
            print("  Checking 10 random samples...")
            import random
            indices = random.sample(range(len(dataset['train'])), min(10, len(dataset['train'])))

            format_errors = 0
            label_errors = 0
            structure_ok = 0

            for idx in indices:
                sample = dataset['train'][idx]['text']

                # Check required tokens
                required_tokens = [
                    '<|begin_of_text|>',
                    '<|start_header_id|>system<|end_header_id|>',
                    '<|start_header_id|>user<|end_header_id|>',
                    '<|start_header_id|>assistant<|end_header_id|>',
                    '<|eot_id|>'
                ]

                missing = [t for t in required_tokens if t not in sample]
                if missing:
                    format_errors += 1
                    continue

                structure_ok += 1

                # Extract assistant response
                if "<|start_header_id|>assistant<|end_header_id|>" in sample:
                    assistant_part = sample.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
                    assistant_response = assistant_part.split("<|eot_id|>")[0].strip()

                    # Validate labels if applicable
                    if expected_labels:
                        if task_key == "ner":
                            # NER: Check entity types
                            lines = assistant_response.split('\n')
                            for line in lines:
                                if ',' in line:
                                    entity_type = line.rsplit(',', 1)[1].strip()
                                    if entity_type not in expected_labels:
                                        label_errors += 1
                                        break
                        else:
                            # Classification: Check first line
                            label = assistant_response.split('\n')[0].strip()
                            if label not in expected_labels:
                                label_errors += 1

            print(f"\n  Results:")
            print(f"    Structure valid: {structure_ok}/10")
            print(f"    Format errors: {format_errors}")
            print(f"    Label errors: {label_errors}")

            if format_errors == 0 and label_errors == 0:
                print(f"\n  [OK] Format validation passed")
                return True
            else:
                print(f"\n  [ERROR] Format validation failed")
                return False

        except Exception as e:
            print(f"  [ERROR] Validation failed: {e}")
            import traceback
            traceback.print_exc()
            return False

    # ========================================================================
    # TEST 3: LABEL DISTRIBUTION
    # ========================================================================

    def test_label_distribution(self, task_key):
        """Test 3: Check label balance and distribution"""
        self.print_section(f"TEST 3: Label Distribution - {task_key.upper()}")

        try:
            dataset = self.loader.load_task_dataset(task_key)

            # Load task config
            with open("data/dataset_config.json", "r") as f:
                task_config = json.load(f)[task_key]

            expected_labels = task_config.get('labels')

            if not expected_labels:
                print("  [SKIP] Not a classification task")
                return True

            # Collect labels from all splits
            for split in dataset.keys():
                print(f"\n  Split: {split}")
                labels_found = []

                for example in dataset[split]:
                    sample = example['text']

                    if "<|start_header_id|>assistant<|end_header_id|>" in sample:
                        assistant_part = sample.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
                        assistant_response = assistant_part.split("<|eot_id|>")[0].strip()

                        if task_key == "ner":
                            # NER: Extract entity types
                            lines = assistant_response.split('\n')
                            for line in lines:
                                if ',' in line:
                                    entity_type = line.rsplit(',', 1)[1].strip()
                                    labels_found.append(entity_type)
                        else:
                            # Classification: First line
                            label = assistant_response.split('\n')[0].strip()
                            labels_found.append(label)

                # Count distribution
                counts = Counter(labels_found)
                total = sum(counts.values())

                print(f"    Total samples: {total}")
                print(f"    Distribution:")
                for label in sorted(expected_labels):
                    count = counts.get(label, 0)
                    percentage = (count / total * 100) if total > 0 else 0
                    print(f"      {label}: {count:,} ({percentage:.1f}%)")

                # Check for severe imbalance
                if total > 0:
                    max_count = max(counts.values())
                    min_count = min(counts.values()) if counts.values() else 0
                    imbalance_ratio = max_count / min_count if min_count > 0 else float('inf')

                    if imbalance_ratio > 10:
                        print(f"    [WARNING] Severe class imbalance detected (ratio: {imbalance_ratio:.1f}:1)")
                    else:
                        print(f"    [OK] Class balance acceptable (ratio: {imbalance_ratio:.1f}:1)")

            return True

        except Exception as e:
            print(f"  [ERROR] Distribution check failed: {e}")
            import traceback
            traceback.print_exc()
            return False

    # ========================================================================
    # TEST 4: TOKENIZATION VALIDATION
    # ========================================================================

    def test_tokenization(self, task_key):
        """Test 4: Validate tokenization limits"""
        self.print_section(f"TEST 4: Tokenization - {task_key.upper()}")

        try:
            # Load tokenizer
            print("  Loading tokenizer...")
            qlora = QLoRAModel()
            tokenizer = qlora.load_tokenizer()

            # Load dataset
            dataset = self.loader.load_task_dataset(task_key)

            # Load task config
            with open(f"configs/tasks/{task_key}_config.yaml", "r") as f:
                task_config = yaml.safe_load(f)

            max_length = task_config['max_sequence_length']

            print(f"  Max sequence length: {max_length}")
            print(f"  Checking 50 random samples...")

            import random
            indices = random.sample(range(len(dataset['train'])), min(50, len(dataset['train'])))

            token_counts = []
            over_limit = 0

            for idx in indices:
                sample = dataset['train'][idx]['text']
                tokens = tokenizer.encode(sample)
                token_count = len(tokens)
                token_counts.append(token_count)

                if token_count > max_length:
                    over_limit += 1

            # Statistics
            import numpy as np
            token_counts = np.array(token_counts)

            print(f"\n  Token statistics:")
            print(f"    Mean: {token_counts.mean():.0f}")
            print(f"    Median: {np.median(token_counts):.0f}")
            print(f"    Min: {token_counts.min()}")
            print(f"    Max: {token_counts.max()}")
            print(f"    95th percentile: {np.percentile(token_counts, 95):.0f}")

            print(f"\n  Samples exceeding limit: {over_limit}/50")

            # Special check for SMP
            if task_key == "smp":
                if token_counts.mean() < 200:
                    print(f"  [WARNING] Samples unexpectedly short for stock movement prediction")
                    print(f"  [WARNING] Expected CSV data and tweets, but average only {token_counts.mean():.0f} tokens")
                    print(f"  [RECOMMENDATION] Verify format_stock_movement() is parsing data correctly")

            if over_limit > 0:
                print(f"  [WARNING] {over_limit} samples exceed max_length")
                print(f"  [RECOMMENDATION] Consider increasing max_sequence_length to {int(np.percentile(token_counts, 95))}")
            else:
                print(f"  [OK] All samples within token limit")

            # Cleanup
            del qlora
            del tokenizer
            torch.cuda.empty_cache()

            return True

        except Exception as e:
            print(f"  [ERROR] Tokenization test failed: {e}")
            import traceback
            traceback.print_exc()
            return False


        # AJOUT 1: Détection d'anomalies
        unique_counts = len(set(token_counts))
        if unique_counts < 10 and len(token_counts) >= 50:
            print(f"  [WARNING] Only {unique_counts} unique token counts")
            print(f"  [WARNING] Suspiciously uniform distribution")

        # AJOUT 2: Percentiles détaillés
        print(f"\n  Detailed percentiles:")
        for p in [25, 50, 75, 90, 95, 99]:
            val = np.percentile(token_counts, p)
            print(f"    {p}th: {val:.0f}")

        # AJOUT 3: Samples proches de la limite
        close_to_limit = sum(1 for t in token_counts if t > max_length * 0.9)
        if close_to_limit > 0:
            print(f"\n  [INFO] {close_to_limit} samples use >90% of max_length")

        # AJOUT 4: Vérification contenu (pour SMP)
        if task_key == "smp":
            print(f"\n  Content validation:")
            sample = dataset['train'][indices[0]]['text']

            has_csv = "Historical Price Data:" in sample
            has_tweets = "Social Media" in sample or "sentiment" in sample.lower()

            print(f"    CSV data present: {'Yes' if has_csv else 'No'}")
            print(f"    Social data present: {'Yes' if has_tweets else 'No'}")

            if not has_csv:
                print(f"  [ERROR] Missing CSV data in samples")
                return False

    # ========================================================================
    # TEST 5: MODEL LOADING (OPTIONAL - HEAVY)
    # ========================================================================

    def test_model_loading(self, task_key, skip=False):
        """Test 5: Verify model can be loaded (can be skipped to save time)"""
        self.print_section(f"TEST 5: Model Loading - {task_key.upper()}")

        if skip:
            print("  [SKIP] Model loading test skipped (use --full to enable)")
            return True

        try:
            # Load task config
            with open(f"configs/tasks/{task_key}_config.yaml", "r") as f:
                task_config = yaml.safe_load(f)

            # Check GPU availability
            if not torch.cuda.is_available():
                print("  [ERROR] CUDA not available. Training requires GPU.")
                return False

            print(f"  GPU: {torch.cuda.get_device_name(0)}")
            print(f"  CUDA Version: {torch.version.cuda}")

            # Check memory
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            print(f"  GPU Memory: {gpu_memory:.1f} GB")

            if gpu_memory < 12:
                print(f"  [WARNING] Low GPU memory. Recommended: 16GB+")
            else:
                print(f"  [OK] Sufficient GPU memory")

            # Try loading model
            print("\n  Loading model with QLoRA (this may take a few minutes)...")
            qlora = QLoRAModel()

            try:
                model = qlora.load_model(task_config)
                print("\n  [OK] Model loaded successfully")

                # Clean up
                del model
                del qlora
                torch.cuda.empty_cache()

                return True

            except Exception as e:
                print(f"\n  [ERROR] Model loading failed: {e}")
                return False

        except Exception as e:
            print(f"  [ERROR] Test failed: {e}")
            import traceback
            traceback.print_exc()
            return False

    # ========================================================================
    # TEST 6: TRAINING CONFIGURATION
    # ========================================================================

    def test_training_config(self, task_key):
        """Test 6: Validate training configuration"""
        self.print_section(f"TEST 6: Training Config - {task_key.upper()}")

        try:
            # Load task config
            with open(f"configs/tasks/{task_key}_config.yaml", "r") as f:
                task_config = yaml.safe_load(f)

            # Display config
            print("  Task Configuration:")
            print(f"    Task name: {task_config['task_name']}")
            print(f"    Adapter name: {task_config['adapter_name']}")
            print(f"    Dataset: {task_config['dataset_path']}")

            print("\n  LoRA Configuration:")
            lora = task_config['lora']
            print(f"    r: {lora['r']}")
            print(f"    alpha: {lora['lora_alpha']}")

            print("\n  Training Configuration:")
            training = task_config['training_args']
            print(f"    Epochs: {training['num_epochs']}")
            print(f"    Learning rate: {training['learning_rate']}")
            print(f"    Batch size: {training['per_device_train_batch_size']}")
            print(f"    Gradient accumulation: {training['gradient_accumulation_steps']}")

            # Calculate effective batch size
            effective_batch = training['per_device_train_batch_size'] * training['gradient_accumulation_steps']
            print(f"    Effective batch size: {effective_batch}")

            # Estimate training steps
            dataset = self.loader.load_task_dataset(task_key)
            train_size = len(dataset['train'])
            steps_per_epoch = train_size // effective_batch
            total_steps = steps_per_epoch * training['num_epochs']

            print(f"\n  Training Estimates:")
            print(f"    Training samples: {train_size:,}")
            print(f"    Steps per epoch: {steps_per_epoch}")
            print(f"    Total steps: {total_steps}")
            print(f"    Eval every: {training['eval_steps']} steps")
            print(f"    Save every: {training['save_steps']} steps")

            # Check if config is reasonable
            warnings = []

            if effective_batch < 8:
                warnings.append("Very small effective batch size")

            if training['learning_rate'] > 1e-3:
                warnings.append("Learning rate might be too high")

            if training['num_epochs'] > 5 and train_size < 1000:
                warnings.append("High epochs for small dataset (overfitting risk)")

            if warnings:
                print(f"\n  [WARNING] Configuration concerns:")
                for w in warnings:
                    print(f"    - {w}")
            else:
                print(f"\n  [OK] Configuration looks reasonable")

            return True

        except Exception as e:
            print(f"  [ERROR] Config validation failed: {e}")
            import traceback
            traceback.print_exc()
            return False

    # ========================================================================
    # TEST 7: SAMPLE INSPECTION
    # ========================================================================

    def test_sample_inspection(self, task_key):
        """Test 7: Inspect actual training samples"""
        self.print_section(f"TEST 7: Sample Inspection - {task_key.upper()}")

        try:
            dataset = self.loader.load_task_dataset(task_key)

            print("  Displaying 2 training samples:\n")

            import random
            indices = random.sample(range(len(dataset['train'])), min(2, len(dataset['train'])))

            for i, idx in enumerate(indices, 1):
                sample = dataset['train'][idx]['text']

                print(f"  {'─'*76}")
                print(f"  SAMPLE {i} (index: {idx})")
                print(f"  {'─'*76}\n")

                # Pretty print with line numbers
                lines = sample.split('\n')
                for line_num, line in enumerate(lines, 1):
                    if line.strip():
                        # Truncate very long lines
                        display_line = line if len(line) <= 70 else line[:67] + "..."
                        print(f"  {line_num:2d} | {display_line}")

                print(f"\n  Length: {len(sample)} characters")
                print()

            return True

        except Exception as e:
            print(f"  [ERROR] Sample inspection failed: {e}")
            import traceback
            traceback.print_exc()
            return False

    # ========================================================================
    # MAIN VALIDATION RUNNER
    # ========================================================================

    def validate_task(self, task_key, skip_model_test=True):
        """Run all validation tests for a task"""
        self.print_header(f"VALIDATING TASK: {task_key.upper()}")

        tests = [
            ("Dataset Integrity", lambda: self.test_dataset_integrity(task_key)),
            ("Format Validation", lambda: self.test_format_validation(task_key)),
            ("Label Distribution", lambda: self.test_label_distribution(task_key)),
            ("Tokenization", lambda: self.test_tokenization(task_key)),
            ("Model Loading", lambda: self.test_model_loading(task_key, skip=skip_model_test)),
            ("Training Config", lambda: self.test_training_config(task_key)),
            ("Sample Inspection", lambda: self.test_sample_inspection(task_key)),
        ]

        results = {}

        for test_name, test_func in tests:
            try:
                passed = test_func()
                results[test_name] = passed
            except Exception as e:
                print(f"\n  [ERROR] Test crashed: {e}")
                import traceback
                traceback.print_exc()
                results[test_name] = False

        # Summary
        self.print_section("TEST SUMMARY")

        passed_count = sum(results.values())
        total_count = len(results)

        for test_name, passed in results.items():
            status = "[PASS]" if passed else "[FAIL]"
            print(f"  {status} {test_name}")

        print(f"\n  Total: {passed_count}/{total_count} tests passed")

        if passed_count == total_count:
            print("\n  [OK] Task is ready for training")
            return True
        else:
            print("\n  [ERROR] Task has validation issues")
            return False

    def validate_all_tasks(self, skip_model_test=True):
        """Validate all tasks"""
        self.print_header("PRE-TRAINING VALIDATION - ALL TASKS")

        # Get available tasks
        tasks = list(self.loader.metadata.keys())

        print(f"Found {len(tasks)} tasks: {', '.join(tasks)}\n")

        overall_results = {}

        for task_key in tasks:
            passed = self.validate_task(task_key, skip_model_test=skip_model_test)
            overall_results[task_key] = passed

        # Final summary
        self.print_header("FINAL SUMMARY")

        for task_key, passed in overall_results.items():
            status = "[READY]" if passed else "[NOT READY]"
            print(f"  {status} {task_key.upper()}")

        passed_count = sum(overall_results.values())
        total_count = len(overall_results)

        print(f"\n  Tasks ready for training: {passed_count}/{total_count}")

        if passed_count == total_count:
            print("\n" + "="*80)
            print("  ALL TASKS READY - YOU CAN START TRAINING")
            print("="*80 + "\n")
        else:
            print("\n" + "="*80)
            print("  SOME TASKS HAVE ISSUES - REVIEW OUTPUT ABOVE")
            print("="*80 + "\n")


# ============================================================================
# USAGE IN NOTEBOOK
# ============================================================================

print("="*80)
print(" PRE-TRAINING VALIDATION TOOL")
print("="*80)
print("\nUsage:")
print("  # Quick validation (skips heavy model loading)")
print("  validator = PreTrainingValidator()")
print("  validator.validate_all_tasks(skip_model_test=True)")
print()
print("  # Full validation (includes model loading - slower)")
print("  validator.validate_all_tasks(skip_model_test=False)")
print()
print("  # Validate single task")
print("  validator.validate_task('sa', skip_model_test=True)")
print("="*80 + "\n")

# Auto-run validation
validator = PreTrainingValidator()
validator.validate_all_tasks(skip_model_test=False)

Working directory: /content/drive/MyDrive/Efficient-Financial-NLP-Fine-Tuning-with-QLoRA
 PRE-TRAINING VALIDATION TOOL

Usage:
  # Quick validation (skips heavy model loading)
  validator = PreTrainingValidator()
  validator.validate_all_tasks(skip_model_test=True)

  # Full validation (includes model loading - slower)
  validator.validate_all_tasks(skip_model_test=False)

  # Validate single task
  validator.validate_task('sa', skip_model_test=True)


 PRE-TRAINING VALIDATION - ALL TASKS

Found 5 tasks: sa, hc, ner, qa, smp


 VALIDATING TASK: SA


────────────────────────────────────────────────────────────────────────────────
  TEST 1: Dataset Integrity - SA
────────────────────────────────────────────────────────────────────────────────

[OK] Loaded: SA
     Path: data/formatted/sa/merged
     Splits: ['train', 'valid', 'test']
     train: 3,849 samples
     valid: 964 samples
     test: 1,205 samples
  [OK] All required splits present
  [OK] All splits contain data

  Split sizes:

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

[OK] Tokenizer loaded
[OK] Loaded: SA
     Path: data/formatted/sa/merged
     Splits: ['train', 'valid', 'test']
     train: 3,849 samples
     valid: 964 samples
     test: 1,205 samples
  Max sequence length: 512
  Checking 50 random samples...

  Token statistics:
    Mean: 71
    Median: 68
    Min: 45
    Max: 110
    95th percentile: 99

  Samples exceeding limit: 0/50
  [OK] All samples within token limit

────────────────────────────────────────────────────────────────────────────────
  TEST 5: Model Loading - SA
────────────────────────────────────────────────────────────────────────────────

  GPU: NVIDIA A100-SXM4-40GB
  CUDA Version: 12.6
  GPU Memory: 39.6 GB
  [OK] Sufficient GPU memory

  Loading model with QLoRA (this may take a few minutes)...

[INFO] Loading model: meta-llama/Meta-Llama-3.1-8B-Instruct

  [ERROR] Model loading failed: No package metadata was found for bitsandbytes

────────────────────────────────────────────────────────────────────────────────
  TES

# ***CELL 6: MULTI-DATASET EVALUATOR***


In [ ]:
"""
================================================================================
CELL 6: MULTI-DATASET EVALUATOR - FINAL CORRECTED VERSION
================================================================================
"""

print("="*80)
print("STEP 6: CREATING EVALUATOR")
print("="*80 + "\n")

evaluator_code = '''"""
Multi-Dataset Evaluator with Rigorous Entity-Level F1 for NER
"""

import torch
import json
import numpy as np
from pathlib import Path
from typing import Dict, List, Optional
import re

from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from tqdm import tqdm

import evaluate

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, matthews_corrcoef
)


def to_native(obj):
    """Convert numpy types to native Python for JSON serialization"""
    if isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: to_native(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [to_native(item) for item in obj]
    return obj


class SOTAComparableEvaluator:
    """Evaluator with rigorous entity-level evaluation"""

    def __init__(self, task_key: str, adapter_path: str,
                 base_model: str = "meta-llama/Meta-Llama-3.1-8B-Instruct",
                 batch_size: int = 8,
                 use_bertscore: bool = True,
                 display_examples: int = 10,
                 live_display: int = 20):

        self.task_key = task_key
        self.adapter_path = Path(adapter_path)
        self.base_model = base_model
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.batch_size = batch_size
        self.use_bertscore = use_bertscore
        self.display_examples = display_examples
        self.live_display = live_display

        with open("data/formatted/metadata.json", "r") as f:
            self.metadata = json.load(f)

        if task_key not in self.metadata:
            raise ValueError(f"Unknown task: {task_key}")

        self.task_info = self.metadata[task_key]
        self.individual_datasets = self.task_info['individual_datasets']

        with open("data/dataset_config.json", "r") as f:
            self.dataset_config = json.load(f)[task_key]

        self.labels = self.dataset_config.get('labels')

        import yaml
        task_config_path = f"configs/tasks/{task_key}_config.yaml"
        with open(task_config_path, "r") as f:
            task_config = yaml.safe_load(f)
        self.max_length = task_config['max_sequence_length']

        self._init_sota_metrics()
        self._load_model()

    def _init_sota_metrics(self):
        """Initialize evaluation metrics"""
        print(f"\\n[INFO] Initializing metrics...")

        if self.task_key == 'qa':
            self.squad_metric = evaluate.load("squad")
            print("  [OK] SQuAD loaded")

        if self.use_bertscore and self.task_key == 'qa':
            self.bertscore = evaluate.load("bertscore")
            print("  [OK] BERTScore loaded")

    def _load_model(self):
        """Load model with adapter"""
        print(f"\\n[INFO] Loading model...")
        print(f"  Base: {self.base_model}")
        print(f"  Adapter: {self.adapter_path}")

        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.tokenizer.padding_side = 'left'

        self.model = AutoModelForCausalLM.from_pretrained(
            self.base_model,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            load_in_4bit=True
        )

        self.model = PeftModel.from_pretrained(self.model, str(self.adapter_path))
        self.model.eval()

        print(f"[OK] Model loaded\\n")

    def _create_inference_prompt(self, formatted_text: str) -> str:
        """Remove ground truth for inference"""
        if "<|start_header_id|>assistant<|end_header_id|>" in formatted_text:
            parts = formatted_text.split("<|start_header_id|>assistant<|end_header_id|>")
            return parts[0] + "<|start_header_id|>assistant<|end_header_id|>"
        return formatted_text

    def extract_ground_truth(self, formatted_text: str) -> str:
        """Extract ground truth"""
        if "<|start_header_id|>assistant<|end_header_id|>" in formatted_text:
            truth = formatted_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
            return truth.split("<|eot_id|>")[0].strip()
        return ""

    def extract_user_input(self, formatted_text: str) -> str:
        """Extract user input for display"""
        if "<|start_header_id|>user<|end_header_id|>" in formatted_text:
            user_part = formatted_text.split("<|start_header_id|>user<|end_header_id|>")[1]
            if "<|eot_id|>" in user_part:
                return user_part.split("<|eot_id|>")[0].strip()
        return ""

    def generate_predictions_batch(self, prompts: List[str], max_new_tokens: int = 50) -> List[str]:
        """Generate predictions"""
        inputs = self.tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=self.max_length
        )

        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )

        responses = []
        for output in outputs:
            full_response = self.tokenizer.decode(output, skip_special_tokens=False)

            if "<|start_header_id|>assistant<|end_header_id|>" in full_response:
                parts = full_response.split("<|start_header_id|>assistant<|end_header_id|>")
                response = parts[-1]

                if "<|eot_id|>" in response:
                    response = response.split("<|eot_id|>")[0]

                response = response.strip()
            else:
                response = self.tokenizer.decode(output, skip_special_tokens=True).strip()

            responses.append(response)

        return responses

    def normalize_prediction(self, text: str) -> Optional[str]:
        """Normalize prediction"""
        if not text:
            return None

        text_lower = text.strip().lower()

        if not self.labels:
            return text.strip()

        if set(self.labels) == {"negative", "neutral", "positive"}:
            if any(x in text_lower for x in ['positive', 'pos', 'bullish', 'good']):
                return "positive"
            elif any(x in text_lower for x in ['negative', 'neg', 'bearish', 'bad']):
                return "negative"
            elif any(x in text_lower for x in ['neutral', 'neut']):
                return "neutral"

        elif set(self.labels) == {"no", "yes"}:
            if any(x in text_lower for x in ['yes', 'y', 'true']):
                return "yes"
            elif any(x in text_lower for x in ['no', 'n', 'false']):
                return "no"

        elif set(self.labels) == {"fall", "rise"}:
            if any(x in text_lower for x in ['rise', 'up', 'increase']):
                return "rise"
            elif any(x in text_lower for x in ['fall', 'down', 'decrease']):
                return "fall"

        return None

    def _display_live_prediction(self, idx: int, dataset_name: str,
                                 user_input: str, raw_output: str,
                                 normalized: str, truth: str):
        """Display prediction in real-time"""
        is_correct = (normalized == self.normalize_prediction(truth))
        symbol = "[OK]" if is_correct else "[FAIL]"

        print(f"\\n{'─'*80}")
        print(f"{symbol} LIVE #{idx+1} - {dataset_name}")
        print(f"{'─'*80}")

        display_input = user_input[:150] + "..." if len(user_input) > 150 else user_input
        print(f"\\nINPUT:\\n  {display_input}")
        print(f"\\nGROUND TRUTH:\\n  {truth}")
        print(f"\\nRAW OUTPUT:\\n  {raw_output}")
        print(f"\\nNORMALIZED:\\n  {normalized}")

        if not is_correct:
            print(f"\\nEXPECTED: {truth}")

    def _display_example(self, idx: int, dataset_name: str, user_input: str,
                        pred: str, truth: str, is_correct: bool, score: float = None):
        """Display evaluation example"""
        status = "CORRECT" if is_correct else "WRONG"
        symbol = "[OK]" if is_correct else "[FAIL]"

        print(f"\\n{'='*80}")
        print(f"{symbol} {dataset_name} - Example {idx+1} - {status}")
        print(f"{'='*80}")

        display_input = user_input[:200] + "..." if len(user_input) > 200 else user_input
        print(f"\\nInput:\\n  {display_input}")
        print(f"\\nGround Truth:\\n  {truth}")
        print(f"\\nPrediction:\\n  {pred}")

        if score is not None:
            print(f"\\nScore: {score:.4f}")

    def evaluate_classification(self, dataset_name: str, dataset, num_samples: int = None):
        """Classification evaluation"""
        test_data = dataset['test']
        if num_samples:
            test_data = test_data.select(range(min(num_samples, len(test_data))))

        y_true = []
        y_pred = []
        y_pred_text = []
        y_true_text = []
        examples_to_display = []

        if self.live_display > 0:
            print(f"\\n{'='*80}")
            print(f"LIVE PREDICTIONS - First {self.live_display}")
            print(f"{'='*80}")

        example_idx = 0
        for i in tqdm(range(0, len(test_data), self.batch_size), desc=f"Evaluating {dataset_name}"):
            batch_indices = range(i, min(i + self.batch_size, len(test_data)))

            prompts = []
            ground_truths = []
            user_inputs = []

            for idx in batch_indices:
                full_text = test_data[idx]['text']
                truth = self.extract_ground_truth(full_text)
                user_input = self.extract_user_input(full_text)
                ground_truths.append(truth)
                user_inputs.append(user_input)

                inference_prompt = self._create_inference_prompt(full_text)
                prompts.append(inference_prompt)

            responses = self.generate_predictions_batch(prompts, max_new_tokens=20)

            for response, truth, user_input in zip(responses, ground_truths, user_inputs):
                pred_label = self.normalize_prediction(response)
                true_label = self.normalize_prediction(truth)

                if example_idx < self.live_display:
                    self._display_live_prediction(
                        example_idx, dataset_name, user_input,
                        response, pred_label, truth
                    )

                if pred_label and true_label:
                    y_pred.append(pred_label)
                    y_true.append(true_label)
                    y_pred_text.append(response)
                    y_true_text.append(truth)

                    is_correct = (pred_label == true_label)

                    if example_idx < self.display_examples:
                        examples_to_display.append({
                            'idx': example_idx,
                            'user_input': user_input,
                            'pred': response,
                            'truth': truth,
                            'is_correct': is_correct
                        })

                example_idx += 1

        if examples_to_display:
            print(f"\\n{'='*80}")
            print(f"FIRST {len(examples_to_display)} EXAMPLES")
            print(f"{'='*80}")

            for ex in examples_to_display:
                self._display_example(
                    ex['idx'], dataset_name, ex['user_input'],
                    ex['pred'], ex['truth'], ex['is_correct']
                )

        label_map = {label: idx for idx, label in enumerate(self.labels)}
        y_true_idx = [label_map[y] for y in y_true]
        y_pred_idx = [label_map[y] for y in y_pred]

        accuracy = accuracy_score(y_true_idx, y_pred_idx)

        if len(self.labels) == 2:
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_true_idx, y_pred_idx, average='binary', zero_division=0
            )
            mcc = matthews_corrcoef(y_true_idx, y_pred_idx)

            metrics = {
                'accuracy': float(accuracy),
                'precision': float(precision),
                'recall': float(recall),
                'f1': float(f1),
                'mcc': float(mcc)
            }
        else:
            precision, recall, f1, support = precision_recall_fscore_support(
                y_true_idx, y_pred_idx, average=None, labels=range(len(self.labels)), zero_division=0
            )
            f1_macro = precision_recall_fscore_support(y_true_idx, y_pred_idx, average='macro', zero_division=0)[2]
            f1_weighted = precision_recall_fscore_support(y_true_idx, y_pred_idx, average='weighted', zero_division=0)[2]
            mcc = matthews_corrcoef(y_true_idx, y_pred_idx)

            metrics = {
                'accuracy': float(accuracy),
                'f1_macro': float(f1_macro),
                'f1_weighted': float(f1_weighted),
                'mcc': float(mcc),
                'per_class': {
                    self.labels[i]: {
                        'precision': float(precision[i]),
                        'recall': float(recall[i]),
                        'f1': float(f1[i]),
                        'support': int(support[i])
                    } for i in range(len(self.labels))
                }
            }

        return metrics

    def evaluate_ner(self, dataset_name: str, dataset, num_samples: int = None):
        """NER evaluation with rigorous entity-level F1"""
        test_data = dataset['test']
        if num_samples:
            test_data = test_data.select(range(min(num_samples, len(test_data))))

        predictions_text = []
        references_text = []
        examples_to_display = []

        if self.live_display > 0:
            print(f"\\n{'='*80}")
            print(f"LIVE PREDICTIONS - First {self.live_display}")
            print(f"{'='*80}")

        example_idx = 0
        for i in tqdm(range(0, len(test_data), self.batch_size), desc=f"Evaluating {dataset_name}"):
            batch_indices = range(i, min(i + self.batch_size, len(test_data)))

            prompts = []
            ground_truths = []
            user_inputs = []

            for idx in batch_indices:
                full_text = test_data[idx]['text']
                truth = self.extract_ground_truth(full_text)
                user_input = self.extract_user_input(full_text)
                ground_truths.append(truth)
                user_inputs.append(user_input)

                inference_prompt = self._create_inference_prompt(full_text)
                prompts.append(inference_prompt)

            responses = self.generate_predictions_batch(prompts, max_new_tokens=200)

            for response, truth, user_input in zip(responses, ground_truths, user_inputs):
                predictions_text.append(response)
                references_text.append(truth)

                pred_entities = self._parse_entities_to_list(response)
                true_entities = self._parse_entities_to_list(truth)

                if example_idx < self.live_display:
                    self._display_live_prediction(
                        example_idx, dataset_name, user_input,
                        response,
                        f"Entities: {len(pred_entities)} (Expected: {len(true_entities)})",
                        truth
                    )

                is_correct = self._entities_match(pred_entities, true_entities)

                if example_idx < self.display_examples:
                    examples_to_display.append({
                        'idx': example_idx,
                        'user_input': user_input,
                        'pred': response,
                        'truth': truth,
                        'is_correct': is_correct
                    })

                example_idx += 1

        if examples_to_display:
            print(f"\\n{'='*80}")
            print(f"FIRST {len(examples_to_display)} EXAMPLES")
            print(f"{'='*80}")

            for ex in examples_to_display:
                self._display_example(
                    ex['idx'], dataset_name, ex['user_input'],
                    ex['pred'], ex['truth'], ex['is_correct']
                )

        entity_metrics = self._calculate_entity_f1(predictions_text, references_text)
        exact_match_acc = self._calculate_exact_match(predictions_text, references_text)
        per_type_metrics = self._calculate_per_entity_type_metrics(predictions_text, references_text)

        metrics = {
            'overall': {
                'entity_f1': entity_metrics['f1'],
                'precision': entity_metrics['precision'],
                'recall': entity_metrics['recall'],
                'exact_match_accuracy': exact_match_acc
            },
            'per_entity_type': per_type_metrics,
            'counts': {
                'true_positives': entity_metrics['true_positives'],
                'false_positives': entity_metrics['false_positives'],
                'false_negatives': entity_metrics['false_negatives']
            },
            'quality_metrics': {
                'avg_duplicates_per_sample': entity_metrics['avg_duplicates'],
                'samples_with_field_labels': entity_metrics['field_label_count'],
                'over_generation_rate': entity_metrics['over_generation_rate']
            }
        }

        return metrics

    def _normalize_entity_name(self, name: str) -> str:
        """Normalize entity name for matching"""
        name = name.lower().strip()
        name = re.sub(r'^\\d+\\s*(/\\s*\\d+)?\\s*', '', name)
        name = re.sub(r'\\b\\d{5}\\b', '', name)
        name = re.sub(r"[',.]", '', name)
        name = re.sub(r'\\s+', ' ', name).strip()
        return name

    def _is_field_label(self, name: str) -> bool:
        """Check if entity is a field label"""
        field_labels = {
            'account number', 'iban number', 'bic number', 'code guichet',
            'no du compte', 'cle rib', 'branch', 'effective date',
            'iban', 'bic', 'swift', 'rib'
        }
        name_lower = name.lower().strip()
        if name_lower in field_labels:
            return True
        if re.match(r'^[A-Z0-9]{6,}$', name):
            return True
        if re.match(r'^\\d+$', name) and len(name) < 8:
            return True
        return False

    def _parse_entities_to_list(self, text: str) -> List[tuple]:
        """Parse NER output to list (preserves duplicates)"""
        entities = []
        seen = {}

        for line in text.strip().split('\\n'):
            line = line.strip()
            if not line or ',' not in line:
                continue

            parts = line.rsplit(',', 1)
            if len(parts) != 2:
                continue

            entity_name = parts[0].strip()
            entity_type = parts[1].strip().upper()

            if entity_type not in ['PER', 'ORG', 'LOC']:
                continue

            if not entity_name or len(entity_name) < 2:
                continue

            if self._is_field_label(entity_name):
                continue

            entity_name_norm = self._normalize_entity_name(entity_name)

            if not entity_name_norm:
                continue

            key = (entity_name_norm, entity_type)
            if key in seen:
                seen[key] += 1
            else:
                seen[key] = 1
                entities.append((entity_name_norm, entity_type))

        return entities

    def _entities_match(self, pred_entities: List[tuple], true_entities: List[tuple]) -> bool:
        """Check if entity lists match (order-invariant)"""
        return set(pred_entities) == set(true_entities)

    def _calculate_entity_f1(self, predictions: List[str], references: List[str]) -> dict:
        """Calculate Entity-Level F1 Score with quality metrics"""
        tp, fp, fn = 0, 0, 0
        total_duplicates = 0
        field_label_count = 0
        total_pred = 0
        total_ref = 0

        for pred_text, ref_text in zip(predictions, references):
            pred_entities_list = self._parse_entities_to_list(pred_text)
            ref_entities_list = self._parse_entities_to_list(ref_text)

            pred_entities = set(pred_entities_list)
            ref_entities = set(ref_entities_list)

            duplicate_count = len(pred_text.strip().split('\\n')) - len(pred_entities)
            if duplicate_count > 0:
                total_duplicates += duplicate_count

            has_field_labels = any(self._is_field_label(line.split(',')[0])
                                  for line in pred_text.strip().split('\\n') if ',' in line)
            if has_field_labels:
                field_label_count += 1

            tp += len(pred_entities & ref_entities)
            fp += len(pred_entities - ref_entities)
            fn += len(ref_entities - pred_entities)

            total_pred += len(pred_entities)
            total_ref += len(ref_entities)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        avg_duplicates = total_duplicates / len(predictions) if predictions else 0
        over_generation_rate = (total_pred - total_ref) / total_ref if total_ref > 0 else 0

        return {
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'true_positives': tp,
            'false_positives': fp,
            'false_negatives': fn,
            'avg_duplicates': avg_duplicates,
            'field_label_count': field_label_count,
            'over_generation_rate': over_generation_rate
        }

    def _calculate_exact_match(self, predictions: List[str], references: List[str]) -> float:
        """Calculate Exact Match Accuracy"""
        correct = 0
        for pred_text, ref_text in zip(predictions, references):
            pred_entities = self._parse_entities_to_list(pred_text)
            ref_entities = self._parse_entities_to_list(ref_text)
            if self._entities_match(pred_entities, ref_entities):
                correct += 1
        return correct / len(predictions) if predictions else 0.0

    def _calculate_per_entity_type_metrics(self, predictions: List[str], references: List[str]) -> dict:
        """Calculate metrics per entity type"""
        type_metrics = {}

        for entity_type in ['PER', 'ORG', 'LOC']:
            tp, fp, fn, support = 0, 0, 0, 0

            for pred_text, ref_text in zip(predictions, references):
                pred_entities = self._parse_entities_to_list(pred_text)
                ref_entities = self._parse_entities_to_list(ref_text)

                pred_type = {(n, t) for n, t in pred_entities if t == entity_type}
                ref_type = {(n, t) for n, t in ref_entities if t == entity_type}

                tp += len(pred_type & ref_type)
                fp += len(pred_type - ref_type)
                fn += len(ref_type - pred_type)
                support += len(ref_type)

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

            type_metrics[entity_type] = {
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'support': support
            }

        return type_metrics

    def calculate_numerical_metrics(self, predictions, references):
        """Calculate numerical accuracy metrics for QA tasks"""

        def extract_number(text):
            text_clean = text.replace(',', '').replace('$', '')
            match = re.search(r'-?\\d+\\.?\\d*', text_clean)
            if match:
                try:
                    return float(match.group())
                except:
                    return None
            return None

        numerical_errors = []
        relative_errors = []
        exact_matches = 0
        within_5_percent = 0
        within_10_percent = 0
        valid_comparisons = 0
        non_numerical = 0

        for pred, ref in zip(predictions, references):
            pred_text = pred['prediction_text']
            ref_text = ref['answers']['text'][0]

            pred_num = extract_number(pred_text)
            ref_num = extract_number(ref_text)

            if pred_num is not None and ref_num is not None and ref_num != 0:
                valid_comparisons += 1

                abs_error = abs(pred_num - ref_num)
                numerical_errors.append(abs_error)

                rel_error = (abs_error / abs(ref_num)) * 100
                relative_errors.append(rel_error)

                if abs_error < 0.01:
                    exact_matches += 1

                if rel_error <= 5.0:
                    within_5_percent += 1
                if rel_error <= 10.0:
                    within_10_percent += 1
            else:
                non_numerical += 1

        if valid_comparisons == 0:
            return None

        return {
            'numerical_exact_match': (exact_matches / valid_comparisons) * 100,
            'within_5_percent': (within_5_percent / valid_comparisons) * 100,
            'within_10_percent': (within_10_percent / valid_comparisons) * 100,
            'mean_absolute_error': sum(numerical_errors) / len(numerical_errors),
            'mean_relative_error_percent': sum(relative_errors) / len(relative_errors),
            'valid_numerical_predictions': valid_comparisons,
            'non_numerical_predictions': non_numerical,
            'total_predictions': len(predictions)
        }

    def evaluate_qa(self, dataset_name: str, dataset, num_samples: int = None):
        """QA evaluation with both official (EM) and numerical metrics"""
        test_data = dataset['test']
        if num_samples:
            test_data = test_data.select(range(min(num_samples, len(test_data))))

        predictions = []
        references = []
        pred_texts = []
        ref_texts = []
        examples_to_display = []

        if self.live_display > 0:
            print(f"\\n{'='*80}")
            print(f"LIVE PREDICTIONS - First {self.live_display}")
            print(f"{'='*80}")

        example_idx = 0
        for i in tqdm(range(0, len(test_data), self.batch_size), desc=f"Evaluating {dataset_name}"):
            batch_indices = range(i, min(i + self.batch_size, len(test_data)))

            prompts = []
            ground_truths = []
            ids = []
            user_inputs = []

            for idx in batch_indices:
                full_text = test_data[idx]['text']
                truth = self.extract_ground_truth(full_text)
                user_input = self.extract_user_input(full_text)
                ground_truths.append(truth)
                user_inputs.append(user_input)

                inference_prompt = self._create_inference_prompt(full_text)
                prompts.append(inference_prompt)
                ids.append(str(idx))

            responses = self.generate_predictions_batch(prompts, max_new_tokens=150)

            for idx, response, truth, user_input in zip(ids, responses, ground_truths, user_inputs):
                if example_idx < self.live_display:
                    self._display_live_prediction(
                        example_idx, dataset_name, user_input,
                        response.strip(), response.strip(), truth.strip()
                    )

                predictions.append({
                    'id': idx,
                    'prediction_text': response.strip()
                })
                references.append({
                    'id': idx,
                    'answers': {
                        'text': [truth.strip()],
                        'answer_start': [0]
                    }
                })
                pred_texts.append(response.strip())
                ref_texts.append(truth.strip())

                individual_result = self.squad_metric.compute(
                    predictions=[predictions[-1]],
                    references=[references[-1]]
                )

                is_correct = (individual_result['exact_match'] == 100.0)
                f1_score = individual_result['f1']

                if example_idx < self.display_examples:
                    examples_to_display.append({
                        'idx': example_idx,
                        'user_input': user_input,
                        'pred': response.strip(),
                        'truth': truth.strip(),
                        'is_correct': is_correct,
                        'score': f1_score / 100.0
                    })

                example_idx += 1

        if examples_to_display:
            print(f"\\n{'='*80}")
            print(f"FIRST {len(examples_to_display)} EXAMPLES")
            print(f"{'='*80}")

            for ex in examples_to_display:
                self._display_example(
                    ex['idx'], dataset_name, ex['user_input'],
                    ex['pred'], ex['truth'], ex['is_correct'], ex['score']
                )

        results = self.squad_metric.compute(
            predictions=predictions,
            references=references
        )

        metrics = {
            'official': {
                'exact_match': results['exact_match'],
                'f1': results['f1']
            }
        }

        print(f"  Computing numerical metrics...")
        numerical_metrics = self.calculate_numerical_metrics(predictions, references)

        if numerical_metrics:
            metrics['numerical'] = numerical_metrics

            print(f"\\n  Numerical Metrics:")
            print(f"    Exact Match (±0.01): {numerical_metrics['numerical_exact_match']:.2f}%")
            print(f"    Within 5%: {numerical_metrics['within_5_percent']:.2f}%")
            print(f"    Within 10%: {numerical_metrics['within_10_percent']:.2f}%")
            print(f"    Mean Abs Error: {numerical_metrics['mean_absolute_error']:.4f}")
            print(f"    Mean Rel Error: {numerical_metrics['mean_relative_error_percent']:.2f}%")
            print(f"    Valid numerical: {numerical_metrics['valid_numerical_predictions']}/{numerical_metrics['total_predictions']}")

        if self.use_bertscore and self.task_key == 'qa':
            print(f"  Computing BERTScore...")

            import gc

            print(f"    [Freeing GPU memory...]")
            self.model.cpu()
            torch.cuda.empty_cache()
            gc.collect()

            try:
                bert_results = self.bertscore.compute(
                    predictions=pred_texts,
                    references=ref_texts,
                    lang="en",
                    model_type="microsoft/deberta-xlarge-mnli",
                    batch_size=8
                )

                metrics['bertscore'] = {
                    'precision': float(np.mean(bert_results['precision'])),
                    'recall': float(np.mean(bert_results['recall'])),
                    'f1': float(np.mean(bert_results['f1']))
                }

                print(f"    [OK] BERTScore computed")

            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"    [WARN] BERTScore skipped (OOM)")
                    metrics['bertscore'] = None
                else:
                    raise

            self.model.to(self.device)

        return metrics

    def evaluate_all_datasets(self, num_samples: int = None):
        """Evaluate all datasets"""
        print(f"\\n{'='*80}")
        print(f"EVALUATION: {self.task_key.upper()}")
        print(f"{'='*80}")
        print(f"\\nTask: {self.task_info['task_name']}")
        print(f"Datasets: {len(self.individual_datasets)}")
        print(f"Display examples: {self.display_examples}")
        print(f"Live display: {self.live_display}")
        print(f"{'='*80}\\n")

        all_results = {}

        for dataset_name, dataset_info in self.individual_datasets.items():
            print(f"\\n{'─'*80}")
            print(f"DATASET: {dataset_name}")
            print(f"{'─'*80}")

            dataset_path = Path(dataset_info['path'])

            if not dataset_path.exists():
                print(f"[ERROR] Dataset not found: {dataset_path}")
                continue

            dataset = load_from_disk(str(dataset_path))

            if 'test' not in dataset:
                print(f"[ERROR] No test split")
                continue

            print(f"Test samples: {len(dataset['test']):,}")

            if self.task_key == 'ner':
                metrics = self.evaluate_ner(dataset_name, dataset, num_samples)
            elif self.task_key == 'qa':
                metrics = self.evaluate_qa(dataset_name, dataset, num_samples)
            else:
                metrics = self.evaluate_classification(dataset_name, dataset, num_samples)

            self._print_metrics(dataset_name, metrics)

            output_dir = Path(f"outputs/evaluations/{self.task_key}")
            output_dir.mkdir(parents=True, exist_ok=True)

            result_file = output_dir / f"{dataset_name}_results.json"
            with open(result_file, 'w') as f:
                json.dump({
                    'dataset': dataset_name,
                    'task': self.task_key,
                    'metrics': to_native(metrics),
                    'samples_evaluated': int(num_samples if num_samples else len(dataset['test']))
                }, f, indent=2)

            print(f"\\n[OK] Results saved: {result_file}")

            all_results[dataset_name] = metrics

        self._print_summary(all_results)

        summary_file = Path(f"outputs/evaluations/{self.task_key}/summary.json")
        with open(summary_file, 'w') as f:
            json.dump({
                'task': self.task_key,
                'task_name': self.task_info['task_name'],
                'adapter': str(self.adapter_path),
                'datasets_evaluated': len(all_results),
                'results': to_native(all_results)
            }, f, indent=2)

        print(f"\\n[OK] Summary saved: {summary_file}")
        print(f"\\n{'='*80}")
        print(f"EVALUATION COMPLETE")
        print(f"{'='*80}\\n")

        return all_results

    def _print_metrics(self, dataset_name: str, metrics: dict):
        """Print metrics"""
        print(f"\\nResults:")

        if 'accuracy' in metrics:
            print(f"  Accuracy: {metrics['accuracy']:.4f}")

        if 'f1' in metrics:
            print(f"  F1: {metrics['f1']:.4f}")

        if 'f1_weighted' in metrics:
            print(f"  F1-Weighted: {metrics['f1_weighted']:.4f}")

        if 'mcc' in metrics:
            print(f"  MCC: {metrics['mcc']:.4f}")

        if 'official' in metrics:
            print(f"  Official Metrics (PIXIU-comparable):")
            print(f"    Exact Match: {metrics['official']['exact_match']:.2f}%")
            print(f"    F1: {metrics['official']['f1']:.2f}%")

        if 'overall' in metrics and 'entity_f1' in metrics['overall']:
            print(f"  Entity-Level Metrics (PIXIU/FinLoRA-comparable):")
            print(f"    Entity F1: {metrics['overall']['entity_f1']:.4f}")
            print(f"    Precision: {metrics['overall']['precision']:.4f}")
            print(f"    Recall: {metrics['overall']['recall']:.4f}")
            print(f"    Exact Match Acc: {metrics['overall']['exact_match_accuracy']:.4f}")

            if 'per_entity_type' in metrics:
                print(f"\\n  Per Entity Type:")
                for etype, type_metrics in metrics['per_entity_type'].items():
                    print(f"    {etype}: F1={type_metrics['f1']:.4f}, P={type_metrics['precision']:.4f}, R={type_metrics['recall']:.4f}, Support={type_metrics['support']}")

            if 'quality_metrics' in metrics:
                qm = metrics['quality_metrics']
                print(f"\\n  Quality Metrics:")
                print(f"    Avg Duplicates/Sample: {qm['avg_duplicates_per_sample']:.2f}")
                print(f"    Samples with Field Labels: {qm['samples_with_field_labels']}")
                print(f"    Over-generation Rate: {qm['over_generation_rate']:.2%}")

        if 'bertscore' in metrics and metrics['bertscore']:
            print(f"  BERTScore F1: {metrics['bertscore']['f1']:.4f}")

    def _print_summary(self, all_results: dict):
        """Print summary"""
        print(f"\\n{'='*80}")
        print(f"SUMMARY - {self.task_key.upper()}")
        print(f"{'='*80}\\n")

        for dataset_name, metrics in all_results.items():
            print(f"{dataset_name}:")

            if 'f1_weighted' in metrics:
                print(f"  F1: {metrics['f1_weighted']:.4f}")
            elif 'f1' in metrics:
                print(f"  F1: {metrics['f1']:.4f}")

            elif 'overall' in metrics and 'entity_f1' in metrics['overall']:
                print(f"  Entity F1: {metrics['overall']['entity_f1']:.4f}")
                print(f"  Exact Match: {metrics['overall']['exact_match_accuracy']:.4f}")

            elif 'official' in metrics:
                print(f"  Official EM: {metrics['official']['exact_match']:.2f}%")
                if 'numerical' in metrics:
                    num = metrics['numerical']
                    print(f"  Numerical EM (±0.01): {num['numerical_exact_match']:.2f}%")
                    print(f"  Within 5%: {num['within_5_percent']:.2f}%")
                    print(f"  Mean Rel Error: {num['mean_relative_error_percent']:.2f}%")

            if 'accuracy' in metrics:
                print(f"  Accuracy: {metrics['accuracy']:.4f}")

            print()

        print(f"{'='*80}\\n")
'''

with open("src/evaluation/evaluator.py", "w") as f:
    f.write(evaluator_code)

print("[OK] src/evaluation/evaluator.py created")
print(" Rigorous NER evaluation with:")
print("   - Normalized entity matching")
print("   - Field label filtering")
print("   - Duplication detection")
print("   - Over-generation tracking")

STEP 6: CREATING EVALUATOR

[OK] src/evaluation/evaluator.py created
 Rigorous NER evaluation with:
   - Normalized entity matching
   - Field label filtering
   - Duplication detection
   - Over-generation tracking


In [ ]:
"""
================================================================================
REAL-WORLD TOKENIZATION & MODEL INPUT TEST
================================================================================
Test sur les VRAIES données pour voir exactement ce que le modèle va recevoir
"""

print("="*80)
print("REAL-WORLD TOKENIZATION & MODEL INPUT VALIDATION")
print("="*80 + "\n")

import json
import torch
from pathlib import Path
from datasets import load_from_disk
from transformers import AutoTokenizer
import numpy as np
from collections import defaultdict

# ============================================================================
# SETUP
# ============================================================================

print("Loading tokenizer and configurations...")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

with open("data/dataset_config.json", "r") as f:
    dataset_config = json.load(f)

with open("data/formatted/metadata.json", "r") as f:
    metadata = json.load(f)

print(f"[OK] Tokenizer loaded")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  Pad token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")
print(f"  EOS token: '{tokenizer.eos_token}' (ID: {tokenizer.eos_token_id})")
print()


# ============================================================================
# TEST 1: REAL TOKENIZATION ON ALL TASKS
# ============================================================================

print("="*80)
print("TEST 1: REAL TOKENIZATION ANALYSIS (ALL TASKS)")
print("="*80 + "\n")

all_tokenization_results = {}

for task in ["sa", "hc", "ner", "qa", "smp"]:
    print(f"\n{'='*70}")
    print(f"TASK: {task.upper()}")
    print(f"{'='*70}\n")

    # Load dataset
    merged_path = Path(metadata[task]['merged_path'])
    dataset = load_from_disk(str(merged_path))

    max_length = dataset_config[task]['max_length']

    task_results = {
        'max_length': max_length,
        'splits': {}
    }

    for split_name in ['train', 'valid', 'test']:
        if split_name not in dataset:
            continue

        print(f"\n{split_name.upper()} SPLIT:")
        print("─" * 60)

        split_data = dataset[split_name]
        print(f"Total samples: {len(split_data):,}")

        # Tokenize ALL samples (or sample if too large)
        num_to_check = min(1000, len(split_data))  # Check up to 1000 samples
        print(f"Analyzing {num_to_check} samples...\n")

        all_token_counts = []
        over_limit_examples = []
        special_token_counts = defaultdict(int)

        for idx in range(num_to_check):
            sample_text = split_data[idx]['text']

            # Tokenize EXACTLY as trainer will
            encoded = tokenizer(
                sample_text,
                truncation=False,  # First without truncation to see real lengths
                add_special_tokens=True,
                return_tensors=None
            )

            token_ids = encoded['input_ids']
            token_count = len(token_ids)
            all_token_counts.append(token_count)

            # Count special tokens
            for token_id in token_ids:
                if token_id in [tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.bos_token_id]:
                    special_token_counts[tokenizer.decode([token_id])] += 1

            # Track over-limit examples
            if token_count > max_length:
                over_limit_examples.append({
                    'idx': idx,
                    'token_count': token_count,
                    'overflow': token_count - max_length,
                    'text_preview': sample_text[:100]
                })

        # Statistics
        token_array = np.array(all_token_counts)

        split_results = {
            'samples_analyzed': num_to_check,
            'total_samples': len(split_data),
            'token_stats': {
                'min': int(np.min(token_array)),
                'max': int(np.max(token_array)),
                'mean': float(np.mean(token_array)),
                'median': float(np.median(token_array)),
                'std': float(np.std(token_array)),
                'p25': float(np.percentile(token_array, 25)),
                'p50': float(np.percentile(token_array, 50)),
                'p75': float(np.percentile(token_array, 75)),
                'p90': float(np.percentile(token_array, 90)),
                'p95': float(np.percentile(token_array, 95)),
                'p99': float(np.percentile(token_array, 99)),
                'max_allowed': max_length
            },
            'truncation_impact': {
                'over_limit_count': len(over_limit_examples),
                'over_limit_percentage': 100 * len(over_limit_examples) / num_to_check,
                'examples': over_limit_examples[:3]  # First 3 examples
            }
        }

        # Print results
        stats = split_results['token_stats']
        print("TOKEN LENGTH DISTRIBUTION:")
        print(f"  Min:       {stats['min']:>6}")
        print(f"  25th %ile: {stats['p25']:>6.0f}")
        print(f"  Median:    {stats['p50']:>6.0f}")
        print(f"  Mean:      {stats['mean']:>6.0f}")
        print(f"  75th %ile: {stats['p75']:>6.0f}")
        print(f"  90th %ile: {stats['p90']:>6.0f}")
        print(f"  95th %ile: {stats['p95']:>6.0f}")
        print(f"  99th %ile: {stats['p99']:>6.0f}")
        print(f"  Max:       {stats['max']:>6}")
        print(f"  Max allowed: {stats['max_allowed']:>4}")

        # Truncation impact
        truncation = split_results['truncation_impact']
        over_count = truncation['over_limit_count']
        over_pct = truncation['over_limit_percentage']

        print(f"\nTRUNCATION IMPACT:")
        if over_count > 0:
            print(f"  [WARNING] {over_count}/{num_to_check} samples ({over_pct:.1f}%) exceed max_length")
            print(f"  These will be TRUNCATED during training/evaluation")

            if truncation['examples']:
                print(f"\n  Examples of samples that will be truncated:")
                for ex in truncation['examples']:
                    print(f"    Sample {ex['idx']}: {ex['token_count']} tokens "
                          f"(will lose {ex['overflow']} tokens)")
                    print(f"      Preview: {ex['text_preview']}...")
        else:
            print(f"  [OK] All {num_to_check} samples fit within max_length")

        task_results['splits'][split_name] = split_results

    all_tokenization_results[task] = task_results


# ============================================================================
# TEST 2: DETAILED TOKENIZATION EXAMPLE (ONE SAMPLE PER TASK)
# ============================================================================

print("\n\n" + "="*80)
print("TEST 2: DETAILED TOKENIZATION BREAKDOWN")
print("="*80 + "\n")

print("Showing EXACTLY what the model will receive during training...\n")

for task in ["sa", "ner", "qa"]:  # Representative tasks
    print(f"\n{'='*70}")
    print(f"TASK: {task.upper()}")
    print(f"{'='*70}\n")

    # Load dataset
    merged_path = Path(metadata[task]['merged_path'])
    dataset = load_from_disk(str(merged_path))

    # Get one real training sample
    sample_text = dataset['train'][0]['text']

    print("RAW TEXT (what's in dataset):")
    print("─" * 70)
    print(sample_text[:500])
    if len(sample_text) > 500:
        print(f"\n... (truncated, full length: {len(sample_text)} chars)")
    print("─" * 70)

    # Tokenize
    encoded = tokenizer(
        sample_text,
        truncation=False,
        add_special_tokens=True,
        return_tensors='pt'
    )

    token_ids = encoded['input_ids'][0]
    attention_mask = encoded['attention_mask'][0]

    print(f"\nTOKENIZATION OUTPUT:")
    print(f"  Input IDs shape: {token_ids.shape}")
    print(f"  Total tokens: {len(token_ids)}")
    print(f"  Attention mask shape: {attention_mask.shape}")
    print(f"  Max length for task: {dataset_config[task]['max_length']}")

    if len(token_ids) > dataset_config[task]['max_length']:
        print(f"  [WARNING] Will be truncated to {dataset_config[task]['max_length']} tokens")
        print(f"  Loss of {len(token_ids) - dataset_config[task]['max_length']} tokens")
    else:
        print(f"  [OK] Fits within limit")

    # Show first and last tokens
    print(f"\nFIRST 10 TOKEN IDs:")
    print(f"  {token_ids[:10].tolist()}")
    print(f"  Decoded: {tokenizer.decode(token_ids[:10])[:100]}...")

    print(f"\nLAST 10 TOKEN IDs:")
    print(f"  {token_ids[-10:].tolist()}")
    print(f"  Decoded: {tokenizer.decode(token_ids[-10:])}")

    # Check special tokens
    print(f"\nSPECIAL TOKENS DETECTED:")
    special_found = []
    if tokenizer.bos_token_id and tokenizer.bos_token_id in token_ids:
        special_found.append(f"BOS ({tokenizer.bos_token_id})")
    if tokenizer.eos_token_id in token_ids:
        count = (token_ids == tokenizer.eos_token_id).sum().item()
        special_found.append(f"EOS ({tokenizer.eos_token_id}) x{count}")
    if tokenizer.pad_token_id in token_ids:
        count = (token_ids == tokenizer.pad_token_id).sum().item()
        special_found.append(f"PAD ({tokenizer.pad_token_id}) x{count}")

    if special_found:
        print(f"  Found: {', '.join(special_found)}")
    else:
        print(f"  No special tokens (unexpected!)")


# ============================================================================
# TEST 3: BATCH TOKENIZATION (AS TRAINER WILL DO)
# ============================================================================

print("\n\n" + "="*80)
print("TEST 3: BATCH TOKENIZATION (REALISTIC TRAINING SCENARIO)")
print("="*80 + "\n")

print("Simulating how SFTTrainer will tokenize batches...\n")

task = "sa"
print(f"Task: {task.upper()}")

merged_path = Path(metadata[task]['merged_path'])
dataset = load_from_disk(str(merged_path))

# Get batch of samples (like trainer does)
batch_size = 4
batch_texts = [dataset['train'][i]['text'] for i in range(batch_size)]

print(f"Batch size: {batch_size}")
print(f"Max length: {dataset_config[task]['max_length']}\n")

# Tokenize batch (as trainer does)
encoded_batch = tokenizer(
    batch_texts,
    truncation=True,
    max_length=dataset_config[task]['max_length'],
    padding=True,
    return_tensors='pt'
)

input_ids = encoded_batch['input_ids']
attention_mask = encoded_batch['attention_mask']

print("BATCH TOKENIZATION OUTPUT:")
print(f"  Input IDs shape: {input_ids.shape}")
print(f"  Attention mask shape: {attention_mask.shape}")
print(f"  Expected shape: (batch_size={batch_size}, seq_len<=max_length={dataset_config[task]['max_length']})")

# Analyze each sample in batch
print(f"\nPER-SAMPLE ANALYSIS:")
for i in range(batch_size):
    sample_tokens = input_ids[i]
    sample_mask = attention_mask[i]

    # Count actual tokens (excluding padding)
    actual_tokens = sample_mask.sum().item()
    padding_tokens = (sample_tokens == tokenizer.pad_token_id).sum().item()

    print(f"\n  Sample {i}:")
    print(f"    Total length: {len(sample_tokens)}")
    print(f"    Actual tokens: {actual_tokens}")
    print(f"    Padding tokens: {padding_tokens}")
    print(f"    First 5 tokens: {sample_tokens[:5].tolist()}")
    print(f"    Last 5 tokens: {sample_tokens[-5:].tolist()}")

    # Verify padding is correct
    if padding_tokens > 0:
        # Check if padding is at the end (left padding would be at start)
        is_right_padded = sample_tokens[-1] == tokenizer.pad_token_id
        print(f"    Padding position: {'RIGHT (correct)' if is_right_padded else 'LEFT (check config)'}")


# ============================================================================
# TEST 4: VERIFY SPECIAL TOKEN HANDLING
# ============================================================================

print("\n\n" + "="*80)
print("TEST 4: SPECIAL TOKEN VERIFICATION")
print("="*80 + "\n")

print("Checking that Llama 3.1 special tokens are correctly handled...\n")

# Get a sample
task = "sa"
merged_path = Path(metadata[task]['merged_path'])
dataset = load_from_disk(str(merged_path))
sample_text = dataset['train'][0]['text']

# Tokenize
tokens = tokenizer.encode(sample_text, add_special_tokens=True)

print("SPECIAL TOKEN CHECK:")
print(f"  Total tokens: {len(tokens)}")

# Check for Llama 3.1 format tokens
llama_tokens = [
    '<|begin_of_text|>',
    '<|start_header_id|>',
    '<|end_header_id|>',
    '<|eot_id|>',
]

print(f"\nLlama 3.1 format tokens in vocabulary:")
for token in llama_tokens:
    if token in tokenizer.get_vocab():
        token_id = tokenizer.convert_tokens_to_ids(token)
        is_present = token_id in tokens
        print(f"  {token:25s} ID: {token_id:6d}  Present: {'YES' if is_present else 'NO'}")
    else:
        print(f"  {token:25s} NOT IN VOCAB [ERROR]")

# Decode sample to verify
print(f"\nDECODED SAMPLE (first 200 chars):")
decoded = tokenizer.decode(tokens[:50])  # First 50 tokens
print(f"  {decoded[:200]}...")


# ============================================================================
# TEST 5: ISSUE DETECTION
# ============================================================================

print("\n\n" + "="*80)
print("TEST 5: POTENTIAL ISSUES DETECTION")
print("="*80 + "\n")

issues = []

# Check 1: Excessive truncation
for task, results in all_tokenization_results.items():
    for split_name, split_data in results['splits'].items():
        over_pct = split_data['truncation_impact']['over_limit_percentage']
        if over_pct > 10:  # More than 10% truncated
            issues.append(f"[CRITICAL] {task.upper()} {split_name}: {over_pct:.1f}% samples will be truncated")
        elif over_pct > 5:  # 5-10% truncated
            issues.append(f"[WARNING] {task.upper()} {split_name}: {over_pct:.1f}% samples will be truncated")

# Check 2: Verify pad token is set
if tokenizer.pad_token_id is None:
    issues.append("[CRITICAL] Pad token is not set!")

# Check 3: Verify EOS token
if tokenizer.eos_token_id is None:
    issues.append("[CRITICAL] EOS token is not set!")

# Report
if issues:
    print("ISSUES DETECTED:")
    for issue in issues:
        print(f"  {issue}")
else:
    print("[OK] No critical issues detected")


# ============================================================================
# FINAL REPORT
# ============================================================================

print("\n\n" + "="*80)
print("REAL-WORLD TOKENIZATION REPORT")
print("="*80 + "\n")

print("SUMMARY BY TASK:")
print("─" * 80)

for task, results in all_tokenization_results.items():
    print(f"\n{task.upper()}:")
    print(f"  Max length: {results['max_length']}")

    for split_name, split_data in results['splits'].items():
        stats = split_data['token_stats']
        truncation = split_data['truncation_impact']

        status = "OK" if truncation['over_limit_percentage'] == 0 else \
                 "WARNING" if truncation['over_limit_percentage'] < 10 else "CRITICAL"

        symbol = "[OK]" if status == "OK" else \
                 "[WARN]" if status == "WARNING" else "[ERROR]"

        print(f"  {symbol} {split_name:5s}: mean={stats['mean']:>6.0f}, "
              f"p95={stats['p95']:>6.0f}, max={stats['max']:>6}, "
              f"truncated={truncation['over_limit_percentage']:.1f}%")

print("\n" + "─" * 80)

# Overall verdict
all_ok = all(
    split_data['truncation_impact']['over_limit_percentage'] < 10
    for task_results in all_tokenization_results.values()
    for split_data in task_results['splits'].values()
)

if all_ok and not issues:
    print("\n[PASS] TOKENIZATION VERIFIED ON REAL DATA")
    print("\nTokenization will work correctly during training:")
    print("  - All samples tokenize properly")
    print("  - Special tokens are handled correctly")
    print("  - Truncation is minimal (<10% samples)")
    print("  - Padding works as expected")
    print("\nCONFIDENCE: 100% - READY FOR TRAINING")
else:
    print("\n[WARNING] TOKENIZATION HAS ISSUES")
    print("\nReview issues above before training")
    print("Consider:")
    print("  - Increasing max_length for affected tasks")
    print("  - Accepting truncation if <10% samples affected")

print("\n" + "="*80)

REAL-WORLD TOKENIZATION & MODEL INPUT VALIDATION

Loading tokenizer and configurations...


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

[OK] Tokenizer loaded
  Vocab size: 128256
  Pad token: '<|eot_id|>' (ID: 128009)
  EOS token: '<|eot_id|>' (ID: 128009)

TEST 1: REAL TOKENIZATION ANALYSIS (ALL TASKS)


TASK: SA


TRAIN SPLIT:
────────────────────────────────────────────────────────────
Total samples: 3,849
Analyzing 1000 samples...

TOKEN LENGTH DISTRIBUTION:
  Min:           50
  25th %ile:     63
  Median:        70
  Mean:          73
  75th %ile:     80
  90th %ile:     94
  95th %ile:    101
  99th %ile:    112
  Max:          124
  Max allowed:  512

TRUNCATION IMPACT:
  [OK] All 1000 samples fit within max_length

VALID SPLIT:
────────────────────────────────────────────────────────────
Total samples: 964
Analyzing 964 samples...

TOKEN LENGTH DISTRIBUTION:
  Min:           41
  25th %ile:     59
  Median:        68
  Mean:          70
  75th %ile:     78
  90th %ile:     92
  95th %ile:    101
  99th %ile:    112
  Max:          137
  Max allowed:  512

TRUNCATION IMPACT:
  [OK] All 964 samples fit within ma

# ***CELL 7: EXECUTION SCRIPTS***

In [ ]:
"""
================================================================================
CELL 7: TRAINING AND EVALUATION SCRIPTS
================================================================================
Creates executable scripts for training and evaluation
"""

print("="*80)
print("STEP 7: CREATING TRAINING AND EVALUATION SCRIPTS")
print("="*80 + "\n")

# ============================================================================
# 7.1: TRAINING SCRIPT
# ============================================================================

training_script = '''#!/usr/bin/env python3
"""
Training Script
Execute training for a specific task

Usage:
    python scripts/train.py --task sa
    python scripts/train.py --task ner
    python scripts/train.py --task qa --epochs 5
"""

import argparse
import sys
from pathlib import Path

# Add src to Python path
import os
import sys
from pathlib import Path

# Ensure we're at project root if running from "scripts"
if 'scripts' in os.getcwd():
    os.chdir('..')

# Add the src directory to sys.path
project_root = Path(__file__).resolve().parent.parent
sys.path.insert(0, str(project_root / "src"))

from src.training.trainer import TaskTrainer


def main():
    parser = argparse.ArgumentParser(description="Train QLoRA adapter for financial tasks")

    parser.add_argument(
        "--task",
        type=str,
        required=True,
        choices=["sa", "hc", "ner", "qa", "smp"],
        help="Task to train (sa, hc, ner, qa, smp)"
    )

    args = parser.parse_args()

    print(f"\\n{'='*80}")
    print(f"TRAINING TASK: {args.task.upper()}")
    print(f"{'='*80}\\n")

    try:
        # Initialize trainer
        trainer = TaskTrainer(task_key=args.task)

        # Train
        model, tokenizer = trainer.train()

        print(f"\\n{'='*80}")
        print("[SUCCESS] Training completed successfully")
        print(f"{'='*80}\\n")

        print("Next steps:")
        print(f"  1. Evaluate: python scripts/evaluate.py --task {args.task}")
        print(f"  2. Check logs: outputs/adapters/{args.task}_adapter/")

    except Exception as e:
        print(f"\\n{'='*80}")
        print("[ERROR] Training failed")
        print(f"{'='*80}")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    main()
'''

with open("scripts/train.py", "w") as f:
    f.write(training_script)

print("[OK] scripts/train.py created")


# ============================================================================
# 7.2: EVALUATION SCRIPT
# ============================================================================

evaluation_script = '''#!/usr/bin/env python3
"""
Evaluation Script
Execute evaluation for a trained adapter

Usage:
    python scripts/evaluate.py --task sa
    python scripts/evaluate.py --task ner --samples 100
    python scripts/evaluate.py --task qa --no-bertscore
"""

import argparse
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path(__file__).parent.parent))

from src.evaluation.evaluator import SOTAComparableEvaluator


def main():
    parser = argparse.ArgumentParser(description="Evaluate trained QLoRA adapter")

    parser.add_argument(
        "--task",
        type=str,
        required=True,
        choices=["sa", "hc", "ner", "qa", "smp"],
        help="Task to evaluate (sa, hc, ner, qa, smp)"
    )

    parser.add_argument(
        "--adapter",
        type=str,
        default=None,
        help="Path to adapter (default: outputs/adapters/{task}_adapter/final_adapter)"
    )

    parser.add_argument(
        "--samples",
        type=int,
        default=None,
        help="Number of samples to evaluate (default: all)"
    )

    parser.add_argument(
        "--batch-size",
        type=int,
        default=8,
        help="Batch size for evaluation (default: 8)"
    )

    parser.add_argument(
        "--no-bertscore",
        action="store_true",
        help="Disable BERTScore computation (faster evaluation)"
    )

    args = parser.parse_args()

    # Default adapter path
    if args.adapter is None:
        args.adapter = f"outputs/adapters/{args.task}_adapter/final_adapter"

    adapter_path = Path(args.adapter)

    if not adapter_path.exists():
        print(f"[ERROR] Adapter not found: {adapter_path}")
        print(f"\\nTrain first with: python scripts/train.py --task {args.task}")
        sys.exit(1)

    print(f"\\n{'='*80}")
    print(f"EVALUATING TASK: {args.task.upper()}")
    print(f"{'='*80}\\n")
    print(f"Adapter: {adapter_path}")
    print(f"Samples: {args.samples if args.samples else 'all'}")
    print(f"Batch size: {args.batch_size}")
    print(f"BERTScore: {not args.no_bertscore}")
    print(f"{'='*80}\\n")

    try:
        # Initialize evaluator
        evaluator = SOTAComparableEvaluator(
            task_key=args.task,
            adapter_path=str(adapter_path),
            batch_size=args.batch_size,
            use_bertscore=not args.no_bertscore
        )

        # Evaluate
        results = evaluator.evaluate_all_datasets(num_samples=args.samples)

        print(f"\\n{'='*80}")
        print("[SUCCESS] Evaluation completed successfully")
        print(f"{'='*80}\\n")

        print("Results saved to:")
        print(f"  outputs/evaluations/{args.task}/")

    except Exception as e:
        print(f"\\n{'='*80}")
        print("[ERROR] Evaluation failed")
        print(f"{'='*80}")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    main()
'''

with open("scripts/evaluate.py", "w") as f:
    f.write(evaluation_script)

print("[OK] scripts/evaluate.py created")


# ============================================================================
# 7.3: VERIFICATION SCRIPT
# ============================================================================

verification_script = '''#!/usr/bin/env python3
"""
Dataset Verification Script
Verify formatted datasets before training

Usage:
    python scripts/verify_datasets.py
    python scripts/verify_datasets.py --task sa
"""

import argparse
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path(__file__).parent.parent))

from src.data.dataset_loader import DatasetLoader


def verify_task(loader, task_key):
    """Verify a single task"""
    print(f"\\n{'='*70}")
    print(f"VERIFYING: {task_key.upper()}")
    print(f"{'='*70}\\n")

    try:
        # Load dataset
        dataset = loader.load_task_dataset(task_key)

        # Validate format
        if loader.validate_format(dataset, task_key):
            print(f"\\n[OK] {task_key.upper()} validation passed\\n")
            return True
        else:
            print(f"\\n[ERROR] {task_key.upper()} validation failed\\n")
            return False

    except Exception as e:
        print(f"\\n[ERROR] Failed to verify {task_key.upper()}")
        print(f"Error: {e}\\n")
        return False


def main():
    parser = argparse.ArgumentParser(description="Verify formatted datasets")

    parser.add_argument(
        "--task",
        type=str,
        default=None,
        choices=["sa", "hc", "ner", "qa", "smp"],
        help="Verify specific task (default: all)"
    )

    args = parser.parse_args()

    print("="*80)
    print("DATASET VERIFICATION")
    print("="*80)

    # Initialize loader
    loader = DatasetLoader()

    # Verify tasks
    if args.task:
        tasks = [args.task]
    else:
        tasks = ["sa", "hc", "ner", "qa", "smp"]

    results = {}
    for task in tasks:
        results[task] = verify_task(loader, task)

    # Summary
    print("="*80)
    print("VERIFICATION SUMMARY")
    print("="*80 + "\\n")

    all_passed = True
    for task, passed in results.items():
        status = "PASS" if passed else "FAIL"
        symbol = "[OK]" if passed else "[ERROR]"
        print(f"  {symbol} {task.upper()}: {status}")
        if not passed:
            all_passed = False

    print("\\n" + "="*80)

    if all_passed:
        print("[SUCCESS] All verifications passed")
        print("="*80 + "\\n")
        print("Ready to train:")
        print("  python scripts/train.py --task sa")
        sys.exit(0)
    else:
        print("[ERROR] Some verifications failed")
        print("="*80 + "\\n")
        print("Fix formatting issues and re-run verification")
        sys.exit(1)


if __name__ == "__main__":
    main()
'''

with open("scripts/verify_datasets.py", "w") as f:
    f.write(verification_script)

print("[OK] scripts/verify_datasets.py created")


# ============================================================================
# 7.4: BATCH TRAINING SCRIPT
# ============================================================================

batch_training_script = '''#!/usr/bin/env python3
"""
Batch Training Script
Train all tasks sequentially

Usage:
    python scripts/train_all.py
    python scripts/train_all.py --tasks sa hc ner
"""

import argparse
import sys
from pathlib import Path
import time

# Add src to path
sys.path.insert(0, str(Path(__file__).parent.parent))

from src.training.trainer import TaskTrainer


def main():
    parser = argparse.ArgumentParser(description="Train all tasks sequentially")

    parser.add_argument(
        "--tasks",
        type=str,
        nargs="+",
        default=["sa", "hc", "ner", "qa", "smp"],
        choices=["sa", "hc", "ner", "qa", "smp"],
        help="Tasks to train (default: all)"
    )

    args = parser.parse_args()

    print("="*80)
    print("BATCH TRAINING")
    print("="*80)
    print(f"\\nTasks to train: {', '.join([t.upper() for t in args.tasks])}")
    print(f"Total tasks: {len(args.tasks)}")
    print("="*80 + "\\n")

    results = {}
    start_time = time.time()

    for i, task in enumerate(args.tasks, 1):
        print(f"\\n{'='*80}")
        print(f"TASK {i}/{len(args.tasks)}: {task.upper()}")
        print(f"{'='*80}\\n")

        task_start = time.time()

        try:
            # Train
            trainer = TaskTrainer(task_key=task)
            model, tokenizer = trainer.train()

            task_time = time.time() - task_start
            results[task] = {
                'status': 'SUCCESS',
                'time': task_time
            }

            print(f"\\n[OK] {task.upper()} completed in {task_time/60:.1f} minutes\\n")

        except Exception as e:
            task_time = time.time() - task_start
            results[task] = {
                'status': 'FAILED',
                'time': task_time,
                'error': str(e)
            }

            print(f"\\n[ERROR] {task.upper()} failed after {task_time/60:.1f} minutes")
            print(f"Error: {e}\\n")

    # Final summary
    total_time = time.time() - start_time

    print("\\n" + "="*80)
    print("BATCH TRAINING SUMMARY")
    print("="*80 + "\\n")

    for task, result in results.items():
        status = result['status']
        task_time = result['time']
        symbol = "[OK]" if status == 'SUCCESS' else "[ERROR]"
        print(f"  {symbol} {task.upper()}: {status} ({task_time/60:.1f}m)")

        if status == 'FAILED':
            print(f"      Error: {result['error']}")

    print(f"\\nTotal time: {total_time/60:.1f} minutes")
    print(f"Total time: {total_time/3600:.2f} hours")
    print("="*80 + "\\n")

    # Check if all succeeded
    all_success = all(r['status'] == 'SUCCESS' for r in results.values())

    if all_success:
        print("[SUCCESS] All tasks trained successfully")
        print("\\nNext steps:")
        print("  python scripts/evaluate_all.py")
    else:
        failed = [t for t, r in results.items() if r['status'] == 'FAILED']
        print(f"[ERROR] {len(failed)} task(s) failed: {', '.join([t.upper() for t in failed])}")
        sys.exit(1)


if __name__ == "__main__":
    main()
'''

with open("scripts/train_all.py", "w") as f:
    f.write(batch_training_script)

print("[OK] scripts/train_all.py created")


# ============================================================================
# 7.5: BATCH EVALUATION SCRIPT
# ============================================================================

batch_evaluation_script = '''#!/usr/bin/env python3
"""
Batch Evaluation Script
Evaluate all trained adapters

Usage:
    python scripts/evaluate_all.py
    python scripts/evaluate_all.py --tasks sa hc ner
    python scripts/evaluate_all.py --samples 100
"""

import argparse
import sys
from pathlib import Path
import time

# Add src to path
sys.path.insert(0, str(Path(__file__).parent.parent))

from src.evaluation.evaluator import SOTAComparableEvaluator


def main():
    parser = argparse.ArgumentParser(description="Evaluate all trained adapters")

    parser.add_argument(
        "--tasks",
        type=str,
        nargs="+",
        default=["sa", "hc", "ner", "qa", "smp"],
        choices=["sa", "hc", "ner", "qa", "smp"],
        help="Tasks to evaluate (default: all)"
    )

    parser.add_argument(
        "--samples",
        type=int,
        default=None,
        help="Number of samples to evaluate per dataset (default: all)"
    )

    parser.add_argument(
        "--batch-size",
        type=int,
        default=8,
        help="Batch size for evaluation (default: 8)"
    )

    parser.add_argument(
        "--no-bertscore",
        action="store_true",
        help="Disable BERTScore computation (faster evaluation)"
    )

    args = parser.parse_args()

    print("="*80)
    print("BATCH EVALUATION")
    print("="*80)
    print(f"\\nTasks to evaluate: {', '.join([t.upper() for t in args.tasks])}")
    print(f"Total tasks: {len(args.tasks)}")
    print(f"Samples per dataset: {args.samples if args.samples else 'all'}")
    print(f"Batch size: {args.batch_size}")
    print(f"BERTScore: {not args.no_bertscore}")
    print("="*80 + "\\n")

    results = {}
    start_time = time.time()

    for i, task in enumerate(args.tasks, 1):
        print(f"\\n{'='*80}")
        print(f"TASK {i}/{len(args.tasks)}: {task.upper()}")
        print(f"{'='*80}\\n")

        adapter_path = f"outputs/adapters/{task}_adapter/final_adapter"

        if not Path(adapter_path).exists():
            print(f"[ERROR] Adapter not found: {adapter_path}")
            print(f"Train first: python scripts/train.py --task {task}\\n")
            results[task] = {'status': 'SKIPPED', 'reason': 'Adapter not found'}
            continue

        task_start = time.time()

        try:
            # Evaluate
            evaluator = SOTAComparableEvaluator(
                task_key=task,
                adapter_path=adapter_path,
                batch_size=args.batch_size,
                use_bertscore=not args.no_bertscore
            )

            task_results = evaluator.evaluate_all_datasets(num_samples=args.samples)

            task_time = time.time() - task_start
            results[task] = {
                'status': 'SUCCESS',
                'time': task_time,
                'results': task_results
            }

            print(f"\\n[OK] {task.upper()} completed in {task_time/60:.1f} minutes\\n")

        except Exception as e:
            task_time = time.time() - task_start
            results[task] = {
                'status': 'FAILED',
                'time': task_time,
                'error': str(e)
            }

            print(f"\\n[ERROR] {task.upper()} failed after {task_time/60:.1f} minutes")
            print(f"Error: {e}\\n")

    # Final summary
    total_time = time.time() - start_time

    print("\\n" + "="*80)
    print("BATCH EVALUATION SUMMARY")
    print("="*80 + "\\n")

    for task, result in results.items():
        status = result['status']

        if status == 'SUCCESS':
            task_time = result['time']
            print(f"  [OK] {task.upper()}: {status} ({task_time/60:.1f}m)")
        elif status == 'SKIPPED':
            print(f"  [SKIP] {task.upper()}: {result['reason']}")
        else:
            task_time = result['time']
            print(f"  [ERROR] {task.upper()}: {status} ({task_time/60:.1f}m)")
            print(f"      Error: {result['error']}")

    print(f"\\nTotal time: {total_time/60:.1f} minutes")
    print(f"Total time: {total_time/3600:.2f} hours")
    print("="*80 + "\\n")

    # Check results
    success_count = sum(1 for r in results.values() if r['status'] == 'SUCCESS')

    if success_count > 0:
        print(f"[SUCCESS] {success_count} task(s) evaluated successfully")
        print("\\nResults saved to: outputs/evaluations/")

    failed = [t for t, r in results.items() if r['status'] == 'FAILED']
    if failed:
        print(f"[ERROR] {len(failed)} task(s) failed: {', '.join([t.upper() for t in failed])}")
        sys.exit(1)


if __name__ == "__main__":
    main()
'''

with open("scripts/evaluate_all.py", "w") as f:
    f.write(batch_evaluation_script)

print("[OK] scripts/evaluate_all.py created")


# ============================================================================
# 7.6: MAKE SCRIPTS EXECUTABLE
# ============================================================================

import os
import stat

scripts = [
    "scripts/train.py",
    "scripts/evaluate.py",
    "scripts/verify_datasets.py",
    "scripts/train_all.py",
    "scripts/evaluate_all.py"
]

for script in scripts:
    path = Path(script)
    if path.exists():
        # Add executable permission
        st = os.stat(path)
        os.chmod(path, st.st_mode | stat.S_IEXEC)

print("\n[OK] Scripts made executable")


print("\n" + "="*80)
print("[OK] ALL SCRIPTS CREATED")
print("="*80 + "\n")

print("AVAILABLE SCRIPTS:")
print("  1. python scripts/verify_datasets.py")
print("     Verify formatted datasets before training")
print()
print("  2. python scripts/train.py --task sa")
print("     Train single task adapter")
print()
print("  3. python scripts/train_all.py")
print("     Train all tasks sequentially")
print()
print("  4. python scripts/evaluate.py --task sa")
print("     Evaluate single task")
print()
print("  5. python scripts/evaluate_all.py")
print("     Evaluate all trained adapters")

STEP 7: CREATING TRAINING AND EVALUATION SCRIPTS

[OK] scripts/train.py created
[OK] scripts/evaluate.py created
[OK] scripts/verify_datasets.py created
[OK] scripts/train_all.py created
[OK] scripts/evaluate_all.py created

[OK] Scripts made executable

[OK] ALL SCRIPTS CREATED

AVAILABLE SCRIPTS:
  1. python scripts/verify_datasets.py
     Verify formatted datasets before training

  2. python scripts/train.py --task sa
     Train single task adapter

  3. python scripts/train_all.py
     Train all tasks sequentially

  4. python scripts/evaluate.py --task sa
     Evaluate single task

  5. python scripts/evaluate_all.py
     Evaluate all trained adapters


# ***CELL 8: DEPENDENCIES & FINAL DOCUMENTATION***


In [ ]:
"""
================================================================================
CELL 8: DOCUMENTATION AND README
================================================================================
"""

print("="*80)
print("STEP 8: CREATING DOCUMENTATION")
print("="*80 + "\n")

# ============================================================================
# 8.1: MAIN README
# ============================================================================

# ============================================================================
# 8.2: REQUIREMENTS.TXT
# ============================================================================

requirements = '''# Core dependencies
torch>=2.1.0
transformers>=4.36.2
datasets>=2.16.1
peft>=0.7.1
trl>=0.8.1
accelerate>=0.25.0
bitsandbytes>=0.41.3

# Evaluation
scikit-learn>=1.3.0
evaluate>=0.4.0
seqeval>=1.2.2
bert-score>=0.3.13

# Utilities
numpy>=1.24.0
pandas>=2.0.0
matplotlib>=3.7.0
pyyaml>=5.9.0
tqdm>=4.65.0

# Optional
jupyter>=1.0.0
#ipywidgets>=8.0.0
'''

with open("requirements.txt", "w") as f:
    f.write(requirements)

print("[OK] requirements.txt created")
# Install command
print("\n📦 Installing dependencies...")
print("   Run: pip install -r requirements.txt\n")
!pip install -r requirements.txt


# ============================================================================
# 8.4: .GITIGNORE
# ============================================================================

gitignore = '''# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
env/
venv/
ENV/
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg

# Jupyter Notebook
.ipynb_checkpoints
*.ipynb

# Environment
.env
.venv

# IDE
.vscode/
.idea/
*.swp
*.swo
*~

# Data (large files)
data/raw/
data/formatted/*/
!data/formatted/.gitkeep

# Model outputs
outputs/adapters/*/
outputs/evaluations/*/
outputs/logs/*/
!outputs/adapters/.gitkeep
!outputs/evaluations/.gitkeep
!outputs/logs/.gitkeep

# Logs
logs/
*.log

# OS
.DS_Store
Thumbs.db

# Temporary files
*.tmp
*.bak
*.swp
'''

with open(".gitignore", "w") as f:
    f.write(gitignore)

print("[OK] .gitignore created")

STEP 8: CREATING DOCUMENTATION

[OK] README.md created
[OK] requirements.txt created

📦 Installing dependencies...
   Run: pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2ef1b85

# Fix

In [ ]:
#!/usr/bin/env python3
"""
AUTOMATIC FIX - Adjust sys.path in all scripts
"""

from pathlib import Path
import re
import sys
import os

print("=" * 80)
print("AUTOMATIC SCRIPT CORRECTION")
print("=" * 80 + "\n")

CORRECT_FIX = '''# ============= FIX: Add project root to Python path =============
project_root = Path(__file__).resolve().parent.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

src_path = project_root / "src"
if not src_path.exists():
    print(f"[ERROR] 'src' directory not found at: {src_path}")
    print(f"[ERROR] Current working directory: {os.getcwd()}")
    print(f"[ERROR] Project root: {project_root}")
    sys.exit(1)
# ============= End of fix =============
'''

scripts_to_fix = [
    "scripts/train.py",
    "scripts/evaluate.py",
    "scripts/verify_datasets.py",
    "scripts/train_all.py",
    "scripts/evaluate_all.py"
]

for script_path in scripts_to_fix:
    path = Path(script_path)

    if not path.exists():
        print(f"[SKIP] {script_path} - File not found")
        continue

    print(f"Processing {script_path}...")

    with open(path, "r") as f:
        content = f.read()

    pattern = r'# ============= FIX:.*?# ============= End of fix =============\n'
    content = re.sub(pattern, '', content, flags=re.DOTALL)

    import_patterns = [
        'from src.training.trainer import TaskTrainer',
        'from src.evaluation.evaluator import SOTAComparableEvaluator',
        'from src.data.dataset_loader import DatasetLoader'
    ]

    inserted = False
    for pattern in import_patterns:
        if pattern in content:
            content = content.replace(pattern, CORRECT_FIX + "\n" + pattern)
            inserted = True
            break

    if inserted:
        with open(path, "w") as f:
            f.write(content)
        print(f"  ✓ Fixed: {script_path}\n")
    else:
        print(f"  ⚠ Warning: No import from src found in {script_path}\n")

print("=" * 80)
print("[DONE] All scripts have been corrected!")
print("=" * 80 + "\n")

"""
FIX: Renommer les scripts d'évaluation pour éviter le conflit
"""

import os
from pathlib import Path

print("="*80)
print("FIXING CIRCULAR IMPORT ISSUE")
print("="*80 + "\n")

# Renommer les fichiers
old_files = {
    "scripts/evaluate.py": "scripts/eval_model.py",
    "scripts/evaluate_all.py": "scripts/eval_all_models.py"
}

for old_path, new_path in old_files.items():
    old = Path(old_path)
    new = Path(new_path)

    if old.exists():
        os.rename(old, new)
        print(f"[OK] Renamed: {old_path} -> {new_path}")
    else:
        print(f"[SKIP] File not found: {old_path}")

print("\n" + "="*80)
print("[OK] FIX APPLIED")
print("="*80 + "\n")

print("New usage:")
print("  python scripts/eval_model.py --task sa")
print("  python scripts/eval_all_models.py")

AUTOMATIC SCRIPT CORRECTION

Processing scripts/train.py...
  ✓ Fixed: scripts/train.py

Processing scripts/evaluate.py...
  ✓ Fixed: scripts/evaluate.py

Processing scripts/verify_datasets.py...
  ✓ Fixed: scripts/verify_datasets.py

Processing scripts/train_all.py...
  ✓ Fixed: scripts/train_all.py

Processing scripts/evaluate_all.py...
  ✓ Fixed: scripts/evaluate_all.py

[DONE] All scripts have been corrected!

FIXING CIRCULAR IMPORT ISSUE

[OK] Renamed: scripts/evaluate.py -> scripts/eval_model.py
[OK] Renamed: scripts/evaluate_all.py -> scripts/eval_all_models.py

[OK] FIX APPLIED

New usage:
  python scripts/eval_model.py --task sa
  python scripts/eval_all_models.py


# ***Running***

# Train

In [ ]:
# Avec lr=0000.2 et 3 epochs
!python scripts/train.py --task sa

Streaming output truncated to the last 5000 lines.
 95% 2298/2410 [05:50<00:17,  6.51it/s]
 95% 2299/2410 [05:50<00:17,  6.52it/s]
 95% 2300/2410 [05:50<00:17,  6.26it/s]
 95% 2301/2410 [05:50<00:17,  6.33it/s]
 96% 2302/2410 [05:51<00:16,  6.41it/s]
 96% 2303/2410 [05:51<00:16,  6.48it/s]
 96% 2304/2410 [05:51<00:16,  6.52it/s]
 96% 2305/2410 [05:51<00:16,  6.49it/s]
 96% 2306/2410 [05:51<00:15,  6.56it/s]
 96% 2307/2410 [05:51<00:15,  6.47it/s]
 96% 2308/2410 [05:52<00:15,  6.47it/s]
 96% 2309/2410 [05:52<00:15,  6.46it/s]
 96% 2310/2410 [05:52<00:15,  6.49it/s]
 96% 2311/2410 [05:52<00:15,  6.51it/s]
 96% 2312/2410 [05:52<00:14,  6.54it/s]
 96% 2313/2410 [05:52<00:14,  6.49it/s]
 96% 2314/2410 [05:52<00:14,  6.50it/s]
 96% 2315/2410 [05:53<00:14,  6.53it/s]
 96% 2316/2410 [05:53<00:14,  6.53it/s]
 96% 2317/2410 [05:53<00:14,  6.55it/s]
 96% 2318/2410 [05:53<00:14,  6.55it/s]
 96% 2319/2410 [05:53<00:13,  6.58it/s]
 96% 2320/2410 [05:53<00:13,  6.57it/s]
 96% 2321/2410 [05:54<00:13, 

In [ ]:
# Avec lr=0000.2 et 5 epochs
!python scripts/train.py --task sa

2025-11-01 19:00:43.386635: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 19:00:43.405189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762023643.427836   14523 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762023643.434507   14523 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762023643.451560   14523 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
# Avec lr=0000.1 et 4 epochs
!python scripts/train.py --task sa

2025-11-01 18:33:48.134574: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 18:33:48.152774: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762022028.174709    7319 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762022028.181610    7319 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762022028.198945    7319 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/train.py --task sa

2025-10-27 11:29:36.090361: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 11:29:36.108298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761564576.129858   15767 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761564576.136332   15767 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761564576.153289   15767 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
# with prompt techniques
!python scripts/train.py --task ner

2025-11-08 19:24:31.611347: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-08 19:24:31.630143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762629871.653446    3719 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762629871.660401    3719 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762629871.677595    3719 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/train.py --task ner

2025-11-01 17:49:48.144215: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 17:49:48.162111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762019388.183966    4751 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762019388.190433    4751 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762019388.206993    4751 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/train.py --task hc

Streaming output truncated to the last 5000 lines.
  8% 211/2568 [00:30<05:48,  6.76it/s]
  8% 212/2568 [00:30<05:55,  6.62it/s]
  8% 213/2568 [00:31<05:49,  6.74it/s]
  8% 214/2568 [00:31<05:49,  6.73it/s]
  8% 215/2568 [00:31<05:46,  6.79it/s]
  8% 216/2568 [00:31<05:48,  6.75it/s]
  8% 217/2568 [00:31<05:49,  6.73it/s]
  8% 218/2568 [00:31<05:43,  6.85it/s]
  9% 219/2568 [00:31<05:56,  6.60it/s]
  9% 220/2568 [00:32<05:49,  6.72it/s]
  9% 221/2568 [00:32<05:52,  6.67it/s]
  9% 222/2568 [00:32<05:45,  6.79it/s]
  9% 223/2568 [00:32<05:52,  6.65it/s]
  9% 224/2568 [00:32<05:45,  6.78it/s]
  9% 225/2568 [00:32<05:47,  6.75it/s]
  9% 226/2568 [00:32<05:48,  6.71it/s]
  9% 227/2568 [00:33<05:43,  6.82it/s]
  9% 228/2568 [00:33<05:46,  6.75it/s]
  9% 229/2568 [00:33<05:41,  6.84it/s]
  9% 230/2568 [00:33<05:44,  6.78it/s]
  9% 231/2568 [00:33<05:40,  6.86it/s]
  9% 232/2568 [00:33<05:45,  6.76it/s]
  9% 233/2568 [00:33<05:39,  6.88it/s]
  9% 234/2568 [00:34<05:46,  6.73it/s]
  9% 235/2568

In [ ]:
!python scripts/train.py --task qa

2025-10-30 09:22:44.576969: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-30 09:22:44.595123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761816164.616706    4980 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761816164.623270    4980 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761816164.639796    4980 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/train.py --task smp

2025-10-31 07:27:31.206563: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 07:27:31.224133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761895651.245251    4731 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761895651.251643    4731 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761895651.267877    4731 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Eval

In [ ]:
!python scripts/eval_model.py --task sa

2025-11-01 18:53:58.118359: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 18:53:58.139171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762023238.162802   12636 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762023238.169629   12636 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762023238.187267   12636 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/eval_model.py --task ner

2025-11-09 12:32:15.772004: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-09 12:32:15.789813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762691535.813229    3989 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762691535.819786    3989 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762691535.836812    3989 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/eval_model.py --task hc

2025-10-31 14:22:22.632316: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 14:22:22.649507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761920542.670887    4671 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761920542.677508    4671 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761920542.693898    4671 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/eval_model.py --task qa

2025-10-30 17:37:10.289943: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-30 17:37:10.308263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761845830.331031   16137 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761845830.337752   16137 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761845830.355128   16137 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/eval_model.py --task smp

2025-10-31 09:31:48.963654: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 09:31:48.981846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761903109.003781   36717 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761903109.010491   36717 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761903109.027151   36717 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# GitHub Push